In [1]:
from tqdm import tqdm
import simplejson as json
import requests
from pyquery import PyQuery as pq
from PIL import Image, ImageOps, ImageEnhance    # Pillow

import os
import urllib.parse
from glob import glob

In [13]:
autocontrastcutoff = 25   # Cut off how many percent of histogram?
imagesharpness = 1.8   # How much to sharpen?  Was 1.5

In [3]:
urlswanted = [
    "https://essss.library.vanderbilt.edu/islandora/object/essss%3A38558#page/1/mode/2up"
]

In [4]:
# rawjson = '[{"pid":"essss:38572","page":"1","label":"DSC08888","width":"1944","height":"2592","dsid":"JP2","token":"d6bf6d2f8fdf59471a7aec8d253fd82c1836b2f292c83ccfa7a5c62e4895c08d","identifier":"essss%3A38572%7EJP2%7Ed6bf6d2f8fdf59471a7aec8d253fd82c1836b2f292c83ccfa7a5c62e4895c08d%7Eessss"},{"pid":"essss:38574","page":"2","label":"DSC08889","width":"2592","height":"1944","dsid":"JP2","token":"3d0b79daefe8bddc0928717f0bd9e02fb9671976e97390d1231be43c887c7e7d","identifier":"essss%3A38574%7EJP2%7E3d0b79daefe8bddc0928717f0bd9e02fb9671976e97390d1231be43c887c7e7d%7Eessss"},{"pid":"essss:38576","page":"3","label":"DSC08890","width":"2592","height":"1944","dsid":"JP2","token":"19e7ca050a72bf085c026bd9bee540290d040a92022cb7ea1a80308b970c3ca0","identifier":"essss%3A38576%7EJP2%7E19e7ca050a72bf085c026bd9bee540290d040a92022cb7ea1a80308b970c3ca0%7Eessss"},{"pid":"essss:38578","page":"4","label":"DSC08891","width":"2592","height":"1944","dsid":"JP2","token":"a52612046d651c9dcfd2ed02e40dcd86f3a8ea48020c19a51865db9ec9f9034b","identifier":"essss%3A38578%7EJP2%7Ea52612046d651c9dcfd2ed02e40dcd86f3a8ea48020c19a51865db9ec9f9034b%7Eessss"},{"pid":"essss:38580","page":"5","label":"DSC08892","width":"2592","height":"1944","dsid":"JP2","token":"35f930f8f749a6fe7df7c599cf16e7d2760a2385b67bf3dda2256aec8a7586d6","identifier":"essss%3A38580%7EJP2%7E35f930f8f749a6fe7df7c599cf16e7d2760a2385b67bf3dda2256aec8a7586d6%7Eessss"},{"pid":"essss:38582","page":"6","label":"DSC08893","width":"2592","height":"1944","dsid":"JP2","token":"f9ce9e3a3a7b714c53ce9297f4ebbcd8e4e290830a36811f637fbd86b5373459","identifier":"essss%3A38582%7EJP2%7Ef9ce9e3a3a7b714c53ce9297f4ebbcd8e4e290830a36811f637fbd86b5373459%7Eessss"},{"pid":"essss:38584","page":"7","label":"DSC08894","width":"2592","height":"1944","dsid":"JP2","token":"0851999873d22d6e72c7255e21bf130a9072ba3949840b910a081005d36f254b","identifier":"essss%3A38584%7EJP2%7E0851999873d22d6e72c7255e21bf130a9072ba3949840b910a081005d36f254b%7Eessss"},{"pid":"essss:38586","page":"8","label":"DSC08895","width":"2592","height":"1944","dsid":"JP2","token":"30e38dac2fb15bfc66ecb18361a9c23f00934b5eb40653acb0a829895933a2a9","identifier":"essss%3A38586%7EJP2%7E30e38dac2fb15bfc66ecb18361a9c23f00934b5eb40653acb0a829895933a2a9%7Eessss"},{"pid":"essss:38588","page":"9","label":"DSC08896","width":"2592","height":"1944","dsid":"JP2","token":"ff3e871278638b7b475b3d5e2ca200b62c85632451de8fa6eac2941ab06366f8","identifier":"essss%3A38588%7EJP2%7Eff3e871278638b7b475b3d5e2ca200b62c85632451de8fa6eac2941ab06366f8%7Eessss"},{"pid":"essss:38590","page":"10","label":"DSC08897","width":"2592","height":"1944","dsid":"JP2","token":"dc27697e675928d0ad410075e78323343b4dde2907cabba280b96b57ffcbbd03","identifier":"essss%3A38590%7EJP2%7Edc27697e675928d0ad410075e78323343b4dde2907cabba280b96b57ffcbbd03%7Eessss"},{"pid":"essss:38592","page":"11","label":"DSC08898","width":"2592","height":"1944","dsid":"JP2","token":"e5f05a37e457e330fa832d8f93833341e840f2e36d02e95e7e0480be2b586614","identifier":"essss%3A38592%7EJP2%7Ee5f05a37e457e330fa832d8f93833341e840f2e36d02e95e7e0480be2b586614%7Eessss"},{"pid":"essss:38594","page":"12","label":"DSC08899","width":"2592","height":"1944","dsid":"JP2","token":"36fc1f1f44403af81e3b00ef8f49756145b9fcecf8ba428450001c57d366109e","identifier":"essss%3A38594%7EJP2%7E36fc1f1f44403af81e3b00ef8f49756145b9fcecf8ba428450001c57d366109e%7Eessss"},{"pid":"essss:38596","page":"13","label":"DSC08900","width":"2592","height":"1944","dsid":"JP2","token":"e4fc3885b7f77b7db0ea9d7b26d41e57b67f6adc05b8aeb1d5e1a9e841e4c030","identifier":"essss%3A38596%7EJP2%7Ee4fc3885b7f77b7db0ea9d7b26d41e57b67f6adc05b8aeb1d5e1a9e841e4c030%7Eessss"},{"pid":"essss:38598","page":"14","label":"DSC08901","width":"2592","height":"1944","dsid":"JP2","token":"0a70f35f909b79c4838e3d27c539e3f0add5ae963a16e642ccd235589b126ec3","identifier":"essss%3A38598%7EJP2%7E0a70f35f909b79c4838e3d27c539e3f0add5ae963a16e642ccd235589b126ec3%7Eessss"},{"pid":"essss:38600","page":"15","label":"DSC08902","width":"2592","height":"1944","dsid":"JP2","token":"54ccb2caefe044c6d2a6fda76a81a8415f514dc6c1c1e46d5d6e1e465c691114","identifier":"essss%3A38600%7EJP2%7E54ccb2caefe044c6d2a6fda76a81a8415f514dc6c1c1e46d5d6e1e465c691114%7Eessss"},{"pid":"essss:38602","page":"16","label":"DSC08903","width":"2592","height":"1944","dsid":"JP2","token":"0be0c7765743b99656ca1efc67f154ae9ae9c0c399279c04e1955cf10531b556","identifier":"essss%3A38602%7EJP2%7E0be0c7765743b99656ca1efc67f154ae9ae9c0c399279c04e1955cf10531b556%7Eessss"},{"pid":"essss:38604","page":"17","label":"DSC08904","width":"2592","height":"1944","dsid":"JP2","token":"60ccb4d8f8a4ddddd46547eaaa8b09fc344bacedfc143df8eb99dc4b2961edbc","identifier":"essss%3A38604%7EJP2%7E60ccb4d8f8a4ddddd46547eaaa8b09fc344bacedfc143df8eb99dc4b2961edbc%7Eessss"},{"pid":"essss:38606","page":"18","label":"DSC08905","width":"2592","height":"1944","dsid":"JP2","token":"1a007fab2f78bccc0d9c8e3888f1dbd8ce160030c5b4789502ba47c5a1a1fed7","identifier":"essss%3A38606%7EJP2%7E1a007fab2f78bccc0d9c8e3888f1dbd8ce160030c5b4789502ba47c5a1a1fed7%7Eessss"},{"pid":"essss:38608","page":"19","label":"DSC08906","width":"2592","height":"1944","dsid":"JP2","token":"a388568cfce4e326e4f82d295e96309914bb6b5369e7dd403aa3a9c6f29703dd","identifier":"essss%3A38608%7EJP2%7Ea388568cfce4e326e4f82d295e96309914bb6b5369e7dd403aa3a9c6f29703dd%7Eessss"},{"pid":"essss:38610","page":"20","label":"DSC08907","width":"2592","height":"1944","dsid":"JP2","token":"0cc865a51205b8b3739871b88bf23babcd4f479659c754b647b82c76c53766a1","identifier":"essss%3A38610%7EJP2%7E0cc865a51205b8b3739871b88bf23babcd4f479659c754b647b82c76c53766a1%7Eessss"},{"pid":"essss:38612","page":"21","label":"DSC08908","width":"2592","height":"1944","dsid":"JP2","token":"8c073f9e98d39c905c5050963cf81da83ca359f38a3e556c93eb82f5fbbd86a5","identifier":"essss%3A38612%7EJP2%7E8c073f9e98d39c905c5050963cf81da83ca359f38a3e556c93eb82f5fbbd86a5%7Eessss"},{"pid":"essss:38614","page":"22","label":"DSC08909","width":"2592","height":"1944","dsid":"JP2","token":"eff499ed29c1a9673c93e100933fa1baa4ebdff269f591af487a4a837d219594","identifier":"essss%3A38614%7EJP2%7Eeff499ed29c1a9673c93e100933fa1baa4ebdff269f591af487a4a837d219594%7Eessss"},{"pid":"essss:38616","page":"23","label":"DSC08910","width":"2592","height":"1944","dsid":"JP2","token":"ce93601d5c49b9742e29d5e781cf0a5019b49206dc0ad096d172e2962de5a8b1","identifier":"essss%3A38616%7EJP2%7Ece93601d5c49b9742e29d5e781cf0a5019b49206dc0ad096d172e2962de5a8b1%7Eessss"},{"pid":"essss:38618","page":"24","label":"DSC08911","width":"2592","height":"1944","dsid":"JP2","token":"aba1638509acafaae5faa847788b1220f5bc76b8727ec4c4a590f73ecd4cb7a5","identifier":"essss%3A38618%7EJP2%7Eaba1638509acafaae5faa847788b1220f5bc76b8727ec4c4a590f73ecd4cb7a5%7Eessss"},{"pid":"essss:38620","page":"25","label":"DSC08912","width":"2592","height":"1944","dsid":"JP2","token":"b89b348ae0088c41dd42dfe9f5b4c3a2a318c60df65c7dae86e4c907ee59ab88","identifier":"essss%3A38620%7EJP2%7Eb89b348ae0088c41dd42dfe9f5b4c3a2a318c60df65c7dae86e4c907ee59ab88%7Eessss"},{"pid":"essss:38622","page":"26","label":"DSC08913","width":"2592","height":"1944","dsid":"JP2","token":"bffcd10b715ce12e7e61d899b3eaeb73f0be752d46ff73bf545cfaed8bffabf6","identifier":"essss%3A38622%7EJP2%7Ebffcd10b715ce12e7e61d899b3eaeb73f0be752d46ff73bf545cfaed8bffabf6%7Eessss"},{"pid":"essss:38624","page":"27","label":"DSC08914","width":"2592","height":"1944","dsid":"JP2","token":"1bc6240ee88473f585a239b195820f1a71fd70c24df862ffe4dc45947823a262","identifier":"essss%3A38624%7EJP2%7E1bc6240ee88473f585a239b195820f1a71fd70c24df862ffe4dc45947823a262%7Eessss"},{"pid":"essss:38625","page":"28","label":"DSC08915","width":"2592","height":"1944","dsid":"JP2","token":"8451d4ea2af30e72a15993b50a08a868e16ccdaefa35c9e0ac2770f3a9895c4d","identifier":"essss%3A38625%7EJP2%7E8451d4ea2af30e72a15993b50a08a868e16ccdaefa35c9e0ac2770f3a9895c4d%7Eessss"},{"pid":"essss:38627","page":"29","label":"DSC08916","width":"2592","height":"1944","dsid":"JP2","token":"9ff647d9366747b36bafef517dd20004cf0560da0e4c52d3c02a2bc2d0dc6724","identifier":"essss%3A38627%7EJP2%7E9ff647d9366747b36bafef517dd20004cf0560da0e4c52d3c02a2bc2d0dc6724%7Eessss"},{"pid":"essss:38628","page":"30","label":"DSC08917","width":"2592","height":"1944","dsid":"JP2","token":"6e1c0e4dc7ddc504b54b5a1d0ebd57f5ae79ced44d073d8857ec4f4a0d7773eb","identifier":"essss%3A38628%7EJP2%7E6e1c0e4dc7ddc504b54b5a1d0ebd57f5ae79ced44d073d8857ec4f4a0d7773eb%7Eessss"},{"pid":"essss:38630","page":"31","label":"DSC08918","width":"2592","height":"1944","dsid":"JP2","token":"1109ebab0246927b57433c4db790bf20fd3db60d2e1ff5cf92edf80cef3a7512","identifier":"essss%3A38630%7EJP2%7E1109ebab0246927b57433c4db790bf20fd3db60d2e1ff5cf92edf80cef3a7512%7Eessss"},{"pid":"essss:38632","page":"32","label":"DSC08919","width":"2592","height":"1944","dsid":"JP2","token":"f745debb5b63d4e3bf250b7a2ad68dfcfbb58bb79ecb2c553d1592fe3bcfe8c7","identifier":"essss%3A38632%7EJP2%7Ef745debb5b63d4e3bf250b7a2ad68dfcfbb58bb79ecb2c553d1592fe3bcfe8c7%7Eessss"},{"pid":"essss:38633","page":"33","label":"DSC08920","width":"2592","height":"1944","dsid":"JP2","token":"046bf794b54922b9354f48bef30b19ad74023dc45d8eb19d3564b943f6e8178a","identifier":"essss%3A38633%7EJP2%7E046bf794b54922b9354f48bef30b19ad74023dc45d8eb19d3564b943f6e8178a%7Eessss"},{"pid":"essss:38635","page":"34","label":"DSC08921","width":"2592","height":"1944","dsid":"JP2","token":"f9339756ba90d793504f73cc0e86c16eb4d17b0f4ca52bd8eb64627adc2ab39c","identifier":"essss%3A38635%7EJP2%7Ef9339756ba90d793504f73cc0e86c16eb4d17b0f4ca52bd8eb64627adc2ab39c%7Eessss"},{"pid":"essss:38637","page":"35","label":"DSC08922","width":"2592","height":"1944","dsid":"JP2","token":"033bb408dc930c7a17f2972d81aaf344b0c461b5756f3bc0782135571a0485a2","identifier":"essss%3A38637%7EJP2%7E033bb408dc930c7a17f2972d81aaf344b0c461b5756f3bc0782135571a0485a2%7Eessss"},{"pid":"essss:38639","page":"36","label":"DSC08923","width":"2592","height":"1944","dsid":"JP2","token":"52bdbb05e195ee5b3c3881a92ce75d7fb284168426be0af32254b58d4843cee8","identifier":"essss%3A38639%7EJP2%7E52bdbb05e195ee5b3c3881a92ce75d7fb284168426be0af32254b58d4843cee8%7Eessss"},{"pid":"essss:38640","page":"37","label":"DSC08924","width":"2592","height":"1944","dsid":"JP2","token":"12d431b406aef81b221db5528ba9a55f0bd879ca72fd698c53b21a72062b4962","identifier":"essss%3A38640%7EJP2%7E12d431b406aef81b221db5528ba9a55f0bd879ca72fd698c53b21a72062b4962%7Eessss"},{"pid":"essss:38642","page":"38","label":"DSC08925","width":"2592","height":"1944","dsid":"JP2","token":"00e574de8c8124386ee03f25e5c57323fadb66b14b0715a1eff59579349e5976","identifier":"essss%3A38642%7EJP2%7E00e574de8c8124386ee03f25e5c57323fadb66b14b0715a1eff59579349e5976%7Eessss"},{"pid":"essss:38644","page":"39","label":"DSC08926","width":"2592","height":"1944","dsid":"JP2","token":"8e2eeeeb0452d2fa56867942945c19734b8ca23ef102443fc29d08915ad454f8","identifier":"essss%3A38644%7EJP2%7E8e2eeeeb0452d2fa56867942945c19734b8ca23ef102443fc29d08915ad454f8%7Eessss"},{"pid":"essss:38646","page":"40","label":"DSC08927","width":"2592","height":"1944","dsid":"JP2","token":"2c0b7ee8b5462562b42eaee787cc860d49dbc9b1ab967a9e86cb299c1208da8f","identifier":"essss%3A38646%7EJP2%7E2c0b7ee8b5462562b42eaee787cc860d49dbc9b1ab967a9e86cb299c1208da8f%7Eessss"},{"pid":"essss:38648","page":"41","label":"DSC08928","width":"2592","height":"1944","dsid":"JP2","token":"8f53fdbf3d1becd8eef334d71ff435ad2c3f9d4b0e5233e383df7793cc37d664","identifier":"essss%3A38648%7EJP2%7E8f53fdbf3d1becd8eef334d71ff435ad2c3f9d4b0e5233e383df7793cc37d664%7Eessss"},{"pid":"essss:38649","page":"42","label":"DSC08929","width":"2592","height":"1944","dsid":"JP2","token":"17ffaca01fe40aa3e246bc0c23d4ec36989c358d463773bfba042eb93ec88632","identifier":"essss%3A38649%7EJP2%7E17ffaca01fe40aa3e246bc0c23d4ec36989c358d463773bfba042eb93ec88632%7Eessss"},{"pid":"essss:38651","page":"43","label":"DSC08930","width":"2592","height":"1944","dsid":"JP2","token":"dd377f83bf4a3b69a45bde2239cc9735bb7ae451de28684617f616d55dcae1ee","identifier":"essss%3A38651%7EJP2%7Edd377f83bf4a3b69a45bde2239cc9735bb7ae451de28684617f616d55dcae1ee%7Eessss"},{"pid":"essss:38653","page":"44","label":"DSC08931","width":"2592","height":"1944","dsid":"JP2","token":"28ca12533ac260a8adcba3fc22d04ac75dc1e50736bb90eb6ebb7a5a8b32fcf9","identifier":"essss%3A38653%7EJP2%7E28ca12533ac260a8adcba3fc22d04ac75dc1e50736bb90eb6ebb7a5a8b32fcf9%7Eessss"},{"pid":"essss:38655","page":"45","label":"DSC08932","width":"2592","height":"1944","dsid":"JP2","token":"335588ca1575ce11678573d3e7a6b99655f9830032d62a815ad63f5a4b8b80c6","identifier":"essss%3A38655%7EJP2%7E335588ca1575ce11678573d3e7a6b99655f9830032d62a815ad63f5a4b8b80c6%7Eessss"},{"pid":"essss:38656","page":"46","label":"DSC08933","width":"2592","height":"1944","dsid":"JP2","token":"855f0feae29efd097c0e4919c8ff86a40743473049ad80a4902ec3ad74732d03","identifier":"essss%3A38656%7EJP2%7E855f0feae29efd097c0e4919c8ff86a40743473049ad80a4902ec3ad74732d03%7Eessss"},{"pid":"essss:38658","page":"47","label":"DSC08934","width":"2592","height":"1944","dsid":"JP2","token":"500fe4646dd2552491cdb41359502d4ecd3cb51748dc33f838aefc0a11232161","identifier":"essss%3A38658%7EJP2%7E500fe4646dd2552491cdb41359502d4ecd3cb51748dc33f838aefc0a11232161%7Eessss"},{"pid":"essss:38660","page":"48","label":"DSC08935","width":"2592","height":"1944","dsid":"JP2","token":"4f20dc07bc17ec8115fda5956e2e0d4ade18d6a4f237edca89cef0fc0fd9f270","identifier":"essss%3A38660%7EJP2%7E4f20dc07bc17ec8115fda5956e2e0d4ade18d6a4f237edca89cef0fc0fd9f270%7Eessss"},{"pid":"essss:38662","page":"49","label":"DSC08936","width":"2592","height":"1944","dsid":"JP2","token":"85ff9e406d3671a99d4bae228168df558ef1de609a4d606b4eaf170211d54a90","identifier":"essss%3A38662%7EJP2%7E85ff9e406d3671a99d4bae228168df558ef1de609a4d606b4eaf170211d54a90%7Eessss"},{"pid":"essss:38663","page":"50","label":"DSC08937","width":"2592","height":"1944","dsid":"JP2","token":"6e7c4b8b5fa63a11b678744e7f47fcb685d361a12675172c98be8368c570375a","identifier":"essss%3A38663%7EJP2%7E6e7c4b8b5fa63a11b678744e7f47fcb685d361a12675172c98be8368c570375a%7Eessss"},{"pid":"essss:38665","page":"51","label":"DSC08938","width":"2592","height":"1944","dsid":"JP2","token":"f7884e208dfc36afda7579c809f610c9c32bd2f471febde0140e27ece7b19349","identifier":"essss%3A38665%7EJP2%7Ef7884e208dfc36afda7579c809f610c9c32bd2f471febde0140e27ece7b19349%7Eessss"},{"pid":"essss:38667","page":"52","label":"DSC08939","width":"2592","height":"1944","dsid":"JP2","token":"450adb62b06c32455fe72c8d15be58a6ceb11795333f3680ada2d3b4bebb0c74","identifier":"essss%3A38667%7EJP2%7E450adb62b06c32455fe72c8d15be58a6ceb11795333f3680ada2d3b4bebb0c74%7Eessss"},{"pid":"essss:38669","page":"53","label":"DSC08940","width":"2592","height":"1944","dsid":"JP2","token":"2bcf94ff1aec2f1c890cdd7d7cc282ad7864dcee87191d71950d8b81b19c8fa1","identifier":"essss%3A38669%7EJP2%7E2bcf94ff1aec2f1c890cdd7d7cc282ad7864dcee87191d71950d8b81b19c8fa1%7Eessss"},{"pid":"essss:38671","page":"54","label":"DSC08941","width":"2592","height":"1944","dsid":"JP2","token":"0011cf22de809d67627d6425879f50b705e0353f026ff3886e70018fd6619f3f","identifier":"essss%3A38671%7EJP2%7E0011cf22de809d67627d6425879f50b705e0353f026ff3886e70018fd6619f3f%7Eessss"},{"pid":"essss:38673","page":"55","label":"DSC08942","width":"2592","height":"1944","dsid":"JP2","token":"de88c4b5abe9ff6cbd7551a7dd761bd64ba557474e559736dc48dbd7ec896e23","identifier":"essss%3A38673%7EJP2%7Ede88c4b5abe9ff6cbd7551a7dd761bd64ba557474e559736dc48dbd7ec896e23%7Eessss"},{"pid":"essss:38675","page":"56","label":"DSC08943","width":"2592","height":"1944","dsid":"JP2","token":"bca2c494b705f594ac74e65a7dce05e14348564b249549540918929f68d9231b","identifier":"essss%3A38675%7EJP2%7Ebca2c494b705f594ac74e65a7dce05e14348564b249549540918929f68d9231b%7Eessss"},{"pid":"essss:38677","page":"57","label":"DSC08944","width":"2592","height":"1944","dsid":"JP2","token":"2fceb0311a229bf805c4a78c48096fb2df24bf14077e2dbc33e9e328fe95da45","identifier":"essss%3A38677%7EJP2%7E2fceb0311a229bf805c4a78c48096fb2df24bf14077e2dbc33e9e328fe95da45%7Eessss"},{"pid":"essss:38678","page":"58","label":"DSC08945","width":"2592","height":"1944","dsid":"JP2","token":"1a5d3b11211af4d04254ef84600ae387f7c0414372a8d80719c35b6f7bd6a0b3","identifier":"essss%3A38678%7EJP2%7E1a5d3b11211af4d04254ef84600ae387f7c0414372a8d80719c35b6f7bd6a0b3%7Eessss"},{"pid":"essss:38680","page":"59","label":"DSC08946","width":"2592","height":"1944","dsid":"JP2","token":"ca10020b583eeca6bcb242b3a160a6ef4567b352a790c24bb46f818248769c27","identifier":"essss%3A38680%7EJP2%7Eca10020b583eeca6bcb242b3a160a6ef4567b352a790c24bb46f818248769c27%7Eessss"},{"pid":"essss:38682","page":"60","label":"DSC08947","width":"2592","height":"1944","dsid":"JP2","token":"cf2c4c258588c03cfb7e51d4a92859a6920d6d5ac85a2ca6d5bc43b06038ceaa","identifier":"essss%3A38682%7EJP2%7Ecf2c4c258588c03cfb7e51d4a92859a6920d6d5ac85a2ca6d5bc43b06038ceaa%7Eessss"},{"pid":"essss:38684","page":"61","label":"DSC08948","width":"2592","height":"1944","dsid":"JP2","token":"ed5dde557e1d1392bf0be533d689dca24e105260ecf68236a9a7c923ed12cc07","identifier":"essss%3A38684%7EJP2%7Eed5dde557e1d1392bf0be533d689dca24e105260ecf68236a9a7c923ed12cc07%7Eessss"},{"pid":"essss:38686","page":"62","label":"DSC08949","width":"2592","height":"1944","dsid":"JP2","token":"d317cc164006fecea453a9dca3c97cf8c4d4b69ce10dbcc4e4f3d285bc890e37","identifier":"essss%3A38686%7EJP2%7Ed317cc164006fecea453a9dca3c97cf8c4d4b69ce10dbcc4e4f3d285bc890e37%7Eessss"},{"pid":"essss:38687","page":"63","label":"DSC08950","width":"2592","height":"1944","dsid":"JP2","token":"9e236bde8fd3530a149ce59ca4aefa07622029aedd7a26aaea703c69f434662a","identifier":"essss%3A38687%7EJP2%7E9e236bde8fd3530a149ce59ca4aefa07622029aedd7a26aaea703c69f434662a%7Eessss"},{"pid":"essss:38688","page":"64","label":"DSC08951","width":"2592","height":"1944","dsid":"JP2","token":"b22b3579e6f9e0fd17b2618756282e17005358e12a2329509600a58423525c8d","identifier":"essss%3A38688%7EJP2%7Eb22b3579e6f9e0fd17b2618756282e17005358e12a2329509600a58423525c8d%7Eessss"},{"pid":"essss:38690","page":"65","label":"DSC08952","width":"2592","height":"1944","dsid":"JP2","token":"534bc72ef41994d1715f036152883d40b637a45dcaf49976ef4793d5562a8a2d","identifier":"essss%3A38690%7EJP2%7E534bc72ef41994d1715f036152883d40b637a45dcaf49976ef4793d5562a8a2d%7Eessss"},{"pid":"essss:38691","page":"66","label":"DSC08953","width":"2592","height":"1944","dsid":"JP2","token":"bd9f47ca2c052059c6d3cebb76ae263f2ece3519b7718013b869884beba60d06","identifier":"essss%3A38691%7EJP2%7Ebd9f47ca2c052059c6d3cebb76ae263f2ece3519b7718013b869884beba60d06%7Eessss"},{"pid":"essss:38693","page":"67","label":"DSC08954","width":"2592","height":"1944","dsid":"JP2","token":"0510c353c6621fa6944256f1c1648a24fcbb78508e6783605d14c6f1678655d2","identifier":"essss%3A38693%7EJP2%7E0510c353c6621fa6944256f1c1648a24fcbb78508e6783605d14c6f1678655d2%7Eessss"},{"pid":"essss:38695","page":"68","label":"DSC08955","width":"2592","height":"1944","dsid":"JP2","token":"f403974878de7b2987f7a877c7cbf41d1ccd6ba2e989b8eee8d0517ced88f7e1","identifier":"essss%3A38695%7EJP2%7Ef403974878de7b2987f7a877c7cbf41d1ccd6ba2e989b8eee8d0517ced88f7e1%7Eessss"},{"pid":"essss:38697","page":"69","label":"DSC08956","width":"2592","height":"1944","dsid":"JP2","token":"e8a7de383f1c96b9faa79a8a9b0fb6bccbcc17899b4d4ee0687a915e413167ca","identifier":"essss%3A38697%7EJP2%7Ee8a7de383f1c96b9faa79a8a9b0fb6bccbcc17899b4d4ee0687a915e413167ca%7Eessss"},{"pid":"essss:38699","page":"70","label":"DSC08957","width":"2592","height":"1944","dsid":"JP2","token":"d6022f22164acf4897efa657aec24c03b2b9938bc0543b62b3b841b2df8899a2","identifier":"essss%3A38699%7EJP2%7Ed6022f22164acf4897efa657aec24c03b2b9938bc0543b62b3b841b2df8899a2%7Eessss"},{"pid":"essss:38701","page":"71","label":"DSC08958","width":"2592","height":"1944","dsid":"JP2","token":"fac19b249706486c1ceb63cc516222c6aa5f7aaa1f668e7b76e13059d7f1e6d6","identifier":"essss%3A38701%7EJP2%7Efac19b249706486c1ceb63cc516222c6aa5f7aaa1f668e7b76e13059d7f1e6d6%7Eessss"},{"pid":"essss:38702","page":"72","label":"DSC08959","width":"2592","height":"1944","dsid":"JP2","token":"4f67f7bd30194f6252c398cd270bd4b59b05b4efd59d106554d6ff0ec5fca5ce","identifier":"essss%3A38702%7EJP2%7E4f67f7bd30194f6252c398cd270bd4b59b05b4efd59d106554d6ff0ec5fca5ce%7Eessss"},{"pid":"essss:38703","page":"73","label":"DSC08960","width":"2592","height":"1944","dsid":"JP2","token":"b6fa25be61e7d0e0dcf49261a8b9cbc59de2bca80b2a374c503eab10e1bb3e38","identifier":"essss%3A38703%7EJP2%7Eb6fa25be61e7d0e0dcf49261a8b9cbc59de2bca80b2a374c503eab10e1bb3e38%7Eessss"},{"pid":"essss:38705","page":"74","label":"DSC08961","width":"2592","height":"1944","dsid":"JP2","token":"fde03812979b6f15371598eb982718bfaf6c05926fa140199661fa6e9d28b6e7","identifier":"essss%3A38705%7EJP2%7Efde03812979b6f15371598eb982718bfaf6c05926fa140199661fa6e9d28b6e7%7Eessss"},{"pid":"essss:38706","page":"75","label":"DSC08962","width":"2592","height":"1944","dsid":"JP2","token":"ca8fc14194190a5d57ca481322cb1f56562ff1707678c077b842fac47f74d583","identifier":"essss%3A38706%7EJP2%7Eca8fc14194190a5d57ca481322cb1f56562ff1707678c077b842fac47f74d583%7Eessss"},{"pid":"essss:38707","page":"76","label":"DSC08963","width":"2592","height":"1944","dsid":"JP2","token":"25041169f10a44993574243972854ffefd55ce07e176279d81bd79ad1149dde9","identifier":"essss%3A38707%7EJP2%7E25041169f10a44993574243972854ffefd55ce07e176279d81bd79ad1149dde9%7Eessss"},{"pid":"essss:38708","page":"77","label":"DSC08964","width":"2592","height":"1944","dsid":"JP2","token":"fa8737281db5581a02ca2d4b8d6b72a82e3e0cd51a45c7cd713bda5354cfbfd6","identifier":"essss%3A38708%7EJP2%7Efa8737281db5581a02ca2d4b8d6b72a82e3e0cd51a45c7cd713bda5354cfbfd6%7Eessss"},{"pid":"essss:38709","page":"78","label":"DSC08965","width":"2592","height":"1944","dsid":"JP2","token":"0c769a951a20e3dce42e94cbc1d42bd1b96aeb06ad690cb2927f1c55929ae262","identifier":"essss%3A38709%7EJP2%7E0c769a951a20e3dce42e94cbc1d42bd1b96aeb06ad690cb2927f1c55929ae262%7Eessss"},{"pid":"essss:38710","page":"79","label":"DSC08966","width":"2592","height":"1944","dsid":"JP2","token":"6b04fad508c05f1d1bd97229c37635d968e7c15a6677e7b93c8f63926c607975","identifier":"essss%3A38710%7EJP2%7E6b04fad508c05f1d1bd97229c37635d968e7c15a6677e7b93c8f63926c607975%7Eessss"},{"pid":"essss:38711","page":"80","label":"DSC08967","width":"2592","height":"1944","dsid":"JP2","token":"2f17b78d6e9de6ba7daee91fd6bba6c73bfb5a213378f74565f8e06449d12267","identifier":"essss%3A38711%7EJP2%7E2f17b78d6e9de6ba7daee91fd6bba6c73bfb5a213378f74565f8e06449d12267%7Eessss"},{"pid":"essss:38712","page":"81","label":"DSC08968","width":"2592","height":"1944","dsid":"JP2","token":"dfbbf240e89e6d1dd1ba30d8c01789641af81b5de615d85e78b0fc62094591d3","identifier":"essss%3A38712%7EJP2%7Edfbbf240e89e6d1dd1ba30d8c01789641af81b5de615d85e78b0fc62094591d3%7Eessss"},{"pid":"essss:38713","page":"82","label":"DSC08969","width":"2592","height":"1944","dsid":"JP2","token":"8710b2e1c2e1c52402cc98f3f9edf4eeea3974ad6664bf0cd39152bd6b0c85d5","identifier":"essss%3A38713%7EJP2%7E8710b2e1c2e1c52402cc98f3f9edf4eeea3974ad6664bf0cd39152bd6b0c85d5%7Eessss"},{"pid":"essss:38714","page":"83","label":"DSC08970","width":"2592","height":"1944","dsid":"JP2","token":"adc8c146320e7d095f6407dde804e60695c797629cd5edfc0081e7973e566b6e","identifier":"essss%3A38714%7EJP2%7Eadc8c146320e7d095f6407dde804e60695c797629cd5edfc0081e7973e566b6e%7Eessss"},{"pid":"essss:38715","page":"84","label":"DSC08971","width":"2592","height":"1944","dsid":"JP2","token":"fdc3b5bf2a1b12957669788fe2cdc0e97242dd89b53d93ca84b27e79bdc6b03e","identifier":"essss%3A38715%7EJP2%7Efdc3b5bf2a1b12957669788fe2cdc0e97242dd89b53d93ca84b27e79bdc6b03e%7Eessss"},{"pid":"essss:38716","page":"85","label":"DSC08972","width":"2592","height":"1944","dsid":"JP2","token":"85c9b11721d7930a4f4df75b5b490f80fbf27301c89f0ccb48f44e50f8821cd0","identifier":"essss%3A38716%7EJP2%7E85c9b11721d7930a4f4df75b5b490f80fbf27301c89f0ccb48f44e50f8821cd0%7Eessss"},{"pid":"essss:38717","page":"86","label":"DSC08973","width":"2592","height":"1944","dsid":"JP2","token":"95d278cf9e8dbd625516fa83830cc29658d2775cbd0b76c1a17dc0b0546d3479","identifier":"essss%3A38717%7EJP2%7E95d278cf9e8dbd625516fa83830cc29658d2775cbd0b76c1a17dc0b0546d3479%7Eessss"},{"pid":"essss:38718","page":"87","label":"DSC08974","width":"2592","height":"1944","dsid":"JP2","token":"9fb3d427cb65839f9bb58367198464bd43052aab88b7d3bf22594481c9e812ab","identifier":"essss%3A38718%7EJP2%7E9fb3d427cb65839f9bb58367198464bd43052aab88b7d3bf22594481c9e812ab%7Eessss"},{"pid":"essss:38719","page":"88","label":"DSC08975","width":"2592","height":"1944","dsid":"JP2","token":"83358efdbe327ebbc7ec4a7636b5c9bee50f4b900bea4b383292749889f580b5","identifier":"essss%3A38719%7EJP2%7E83358efdbe327ebbc7ec4a7636b5c9bee50f4b900bea4b383292749889f580b5%7Eessss"},{"pid":"essss:38720","page":"89","label":"DSC08976","width":"2592","height":"1944","dsid":"JP2","token":"326c42135704abb7d78c9d8ec2bc3f01277a59dbf30bceb87ae63244ee820b68","identifier":"essss%3A38720%7EJP2%7E326c42135704abb7d78c9d8ec2bc3f01277a59dbf30bceb87ae63244ee820b68%7Eessss"},{"pid":"essss:38721","page":"90","label":"DSC08977","width":"2592","height":"1944","dsid":"JP2","token":"1ca9fd7cf3c4380acbb610f968017f6a7e8a4ba5b2ab8a4fdc708f5dbb2bcce5","identifier":"essss%3A38721%7EJP2%7E1ca9fd7cf3c4380acbb610f968017f6a7e8a4ba5b2ab8a4fdc708f5dbb2bcce5%7Eessss"},{"pid":"essss:38722","page":"91","label":"DSC08978","width":"2592","height":"1944","dsid":"JP2","token":"a70e18f8e28dbf754690529adeae33d57fcb6d84fd7dea2bfe75d27a2aded414","identifier":"essss%3A38722%7EJP2%7Ea70e18f8e28dbf754690529adeae33d57fcb6d84fd7dea2bfe75d27a2aded414%7Eessss"},{"pid":"essss:38723","page":"92","label":"DSC08979","width":"2592","height":"1944","dsid":"JP2","token":"6500664da9031da04c0bb3531e1f3e5ac0f5f247701ee4c1b533af3a4a921ef7","identifier":"essss%3A38723%7EJP2%7E6500664da9031da04c0bb3531e1f3e5ac0f5f247701ee4c1b533af3a4a921ef7%7Eessss"},{"pid":"essss:38724","page":"93","label":"DSC08980","width":"2592","height":"1944","dsid":"JP2","token":"9ebd734b7752c160acf44c14b581f8747e83e8a78bf1611f140cc58120980e31","identifier":"essss%3A38724%7EJP2%7E9ebd734b7752c160acf44c14b581f8747e83e8a78bf1611f140cc58120980e31%7Eessss"},{"pid":"essss:38725","page":"94","label":"DSC08981","width":"2592","height":"1944","dsid":"JP2","token":"32130688f4f4656fb2b77ce7832855a18d753955396a5dcd15adebe41e228fe8","identifier":"essss%3A38725%7EJP2%7E32130688f4f4656fb2b77ce7832855a18d753955396a5dcd15adebe41e228fe8%7Eessss"},{"pid":"essss:38726","page":"95","label":"DSC08982","width":"2592","height":"1944","dsid":"JP2","token":"75dd5813940170f817249a13dad6adfa6bb023081ac7687c692fa024ee38b0d4","identifier":"essss%3A38726%7EJP2%7E75dd5813940170f817249a13dad6adfa6bb023081ac7687c692fa024ee38b0d4%7Eessss"},{"pid":"essss:38727","page":"96","label":"DSC08983","width":"2592","height":"1944","dsid":"JP2","token":"c789b38fc946135d60b32af4672c21e4a14d66297cccc8b6d87a1e2997cc1e01","identifier":"essss%3A38727%7EJP2%7Ec789b38fc946135d60b32af4672c21e4a14d66297cccc8b6d87a1e2997cc1e01%7Eessss"},{"pid":"essss:38728","page":"97","label":"DSC08984","width":"2592","height":"1944","dsid":"JP2","token":"8c102fd444042ebf0f389bf98ff60087d1ddf9d0fc950b16e16b8eae17b8117b","identifier":"essss%3A38728%7EJP2%7E8c102fd444042ebf0f389bf98ff60087d1ddf9d0fc950b16e16b8eae17b8117b%7Eessss"},{"pid":"essss:38729","page":"98","label":"DSC08985","width":"2592","height":"1944","dsid":"JP2","token":"5213837c84d3cc9ac00c77081d3fdb22487b69895661eaf3ce0990dd5b8801c4","identifier":"essss%3A38729%7EJP2%7E5213837c84d3cc9ac00c77081d3fdb22487b69895661eaf3ce0990dd5b8801c4%7Eessss"},{"pid":"essss:38730","page":"99","label":"DSC08986","width":"2592","height":"1944","dsid":"JP2","token":"21a69236c6b81e05c850db28cb175ac4394d4bf7c5487de62527efa7fe02e6a4","identifier":"essss%3A38730%7EJP2%7E21a69236c6b81e05c850db28cb175ac4394d4bf7c5487de62527efa7fe02e6a4%7Eessss"},{"pid":"essss:38731","page":"100","label":"DSC08987","width":"2592","height":"1944","dsid":"JP2","token":"4d2af153dcddb49328e551fed68d5f4936964ca0cc6f96eca64df9227b6160d0","identifier":"essss%3A38731%7EJP2%7E4d2af153dcddb49328e551fed68d5f4936964ca0cc6f96eca64df9227b6160d0%7Eessss"},{"pid":"essss:38732","page":"101","label":"DSC08988","width":"2592","height":"1944","dsid":"JP2","token":"69fc4ad00180dbf56a580e08e7c732185a6bf08a6fa49457e5f1eb70a0d82897","identifier":"essss%3A38732%7EJP2%7E69fc4ad00180dbf56a580e08e7c732185a6bf08a6fa49457e5f1eb70a0d82897%7Eessss"},{"pid":"essss:38733","page":"102","label":"DSC08989","width":"2592","height":"1944","dsid":"JP2","token":"00fb7651d6eb1e3ff1ca6b751395a2b0f66f0a26cf36f1bc7c95e102d20d8864","identifier":"essss%3A38733%7EJP2%7E00fb7651d6eb1e3ff1ca6b751395a2b0f66f0a26cf36f1bc7c95e102d20d8864%7Eessss"},{"pid":"essss:38734","page":"103","label":"DSC08990","width":"2592","height":"1944","dsid":"JP2","token":"5ce9bbc23e78e7001f2cf59e56f88a531fb898a3b59f6105ce7095fd0e598724","identifier":"essss%3A38734%7EJP2%7E5ce9bbc23e78e7001f2cf59e56f88a531fb898a3b59f6105ce7095fd0e598724%7Eessss"},{"pid":"essss:38735","page":"104","label":"DSC08991","width":"2592","height":"1944","dsid":"JP2","token":"3ec9320cab742c57fdcdbce246690e40b56db5d7954a07a77ad6fbbd0925e2b8","identifier":"essss%3A38735%7EJP2%7E3ec9320cab742c57fdcdbce246690e40b56db5d7954a07a77ad6fbbd0925e2b8%7Eessss"},{"pid":"essss:38736","page":"105","label":"DSC08992","width":"2592","height":"1944","dsid":"JP2","token":"18be566c84fc510a353c7e46e3de4fc4c70ba42bb4a92a6a80831586fd6af292","identifier":"essss%3A38736%7EJP2%7E18be566c84fc510a353c7e46e3de4fc4c70ba42bb4a92a6a80831586fd6af292%7Eessss"},{"pid":"essss:38737","page":"106","label":"DSC08993","width":"2592","height":"1944","dsid":"JP2","token":"9c9666e113b1813ea30c1fe4d66aee3238a9b89c8e7a95212f850268710f60c5","identifier":"essss%3A38737%7EJP2%7E9c9666e113b1813ea30c1fe4d66aee3238a9b89c8e7a95212f850268710f60c5%7Eessss"},{"pid":"essss:38738","page":"107","label":"DSC08994","width":"2592","height":"1944","dsid":"JP2","token":"7284378c33166f155c3179d71615253c967ffebe6d801a48e7375461cbea6a4d","identifier":"essss%3A38738%7EJP2%7E7284378c33166f155c3179d71615253c967ffebe6d801a48e7375461cbea6a4d%7Eessss"},{"pid":"essss:38739","page":"108","label":"DSC08995","width":"2592","height":"1944","dsid":"JP2","token":"a14d727f62a711fc56b29525c19339fd3e968299c06c7aff09b3d0a63c668371","identifier":"essss%3A38739%7EJP2%7Ea14d727f62a711fc56b29525c19339fd3e968299c06c7aff09b3d0a63c668371%7Eessss"},{"pid":"essss:38740","page":"109","label":"DSC08996","width":"2592","height":"1944","dsid":"JP2","token":"c52486f40ed386f27d40e860c5bc658c707942b72d3c50d1a2b63f7b53a2808b","identifier":"essss%3A38740%7EJP2%7Ec52486f40ed386f27d40e860c5bc658c707942b72d3c50d1a2b63f7b53a2808b%7Eessss"},{"pid":"essss:38741","page":"110","label":"DSC08997","width":"2592","height":"1944","dsid":"JP2","token":"4ebfeddcd59698ab97fbd10df6ce083ea8ead40040c01dc6a9cfd0948ea0dce9","identifier":"essss%3A38741%7EJP2%7E4ebfeddcd59698ab97fbd10df6ce083ea8ead40040c01dc6a9cfd0948ea0dce9%7Eessss"},{"pid":"essss:38742","page":"111","label":"DSC08998","width":"2592","height":"1944","dsid":"JP2","token":"09741da5c60ef38a804d18375d860ac554c8678eb463f6829dbea8755ec171af","identifier":"essss%3A38742%7EJP2%7E09741da5c60ef38a804d18375d860ac554c8678eb463f6829dbea8755ec171af%7Eessss"},{"pid":"essss:38743","page":"112","label":"DSC08999","width":"2592","height":"1944","dsid":"JP2","token":"56549c4038fd826e80d9b348633f54c2ccc0da08dd6bc7cc98a941cade1d013b","identifier":"essss%3A38743%7EJP2%7E56549c4038fd826e80d9b348633f54c2ccc0da08dd6bc7cc98a941cade1d013b%7Eessss"},{"pid":"essss:38744","page":"113","label":"DSC09000","width":"2592","height":"1944","dsid":"JP2","token":"604ef211c61eb441d555f8248af040006311eb3ab7f05c57af0dcad91762db07","identifier":"essss%3A38744%7EJP2%7E604ef211c61eb441d555f8248af040006311eb3ab7f05c57af0dcad91762db07%7Eessss"},{"pid":"essss:38745","page":"114","label":"DSC09001","width":"2592","height":"1944","dsid":"JP2","token":"744b03e52a91345c4683017335e239c66d9bffca57c9b9b9dc57752ac22d6b87","identifier":"essss%3A38745%7EJP2%7E744b03e52a91345c4683017335e239c66d9bffca57c9b9b9dc57752ac22d6b87%7Eessss"},{"pid":"essss:38746","page":"115","label":"DSC09002","width":"2592","height":"1944","dsid":"JP2","token":"f768ffc72b40f0728d7fdc1ea11a9bb078d4b19ff045803f959f91fe0c58849f","identifier":"essss%3A38746%7EJP2%7Ef768ffc72b40f0728d7fdc1ea11a9bb078d4b19ff045803f959f91fe0c58849f%7Eessss"},{"pid":"essss:38747","page":"116","label":"DSC09003","width":"2592","height":"1944","dsid":"JP2","token":"43befa4fab3048a10baca9589bf3463115db7516d976b8126005b069b775b926","identifier":"essss%3A38747%7EJP2%7E43befa4fab3048a10baca9589bf3463115db7516d976b8126005b069b775b926%7Eessss"},{"pid":"essss:38748","page":"117","label":"DSC09004","width":"2592","height":"1944","dsid":"JP2","token":"3cb6d1f5204f6ac329d52a97ed17bcfeff75cd2c73de9df92a1a8d1b5587d361","identifier":"essss%3A38748%7EJP2%7E3cb6d1f5204f6ac329d52a97ed17bcfeff75cd2c73de9df92a1a8d1b5587d361%7Eessss"},{"pid":"essss:38749","page":"118","label":"DSC09005","width":"2592","height":"1944","dsid":"JP2","token":"9533fdb9253e6eaa24342be4f996257cc93e834e220b4baac1183672afb94a78","identifier":"essss%3A38749%7EJP2%7E9533fdb9253e6eaa24342be4f996257cc93e834e220b4baac1183672afb94a78%7Eessss"},{"pid":"essss:38750","page":"119","label":"DSC09006","width":"2592","height":"1944","dsid":"JP2","token":"37b4a0aff5488ecfdddcbf5a0dbc68fcd727bec5ea828794a127bc47f7cb167d","identifier":"essss%3A38750%7EJP2%7E37b4a0aff5488ecfdddcbf5a0dbc68fcd727bec5ea828794a127bc47f7cb167d%7Eessss"},{"pid":"essss:38751","page":"120","label":"DSC09007","width":"2592","height":"1944","dsid":"JP2","token":"be33800eae388792c67f236cf5b8ee79126701519ae4fe0cd81f1a844bc3c885","identifier":"essss%3A38751%7EJP2%7Ebe33800eae388792c67f236cf5b8ee79126701519ae4fe0cd81f1a844bc3c885%7Eessss"},{"pid":"essss:38752","page":"121","label":"DSC09008","width":"2592","height":"1944","dsid":"JP2","token":"39f58923be8b44bed98686c142fb9cd2034d9bef19ef0b3e8b4de24c86f5ca8a","identifier":"essss%3A38752%7EJP2%7E39f58923be8b44bed98686c142fb9cd2034d9bef19ef0b3e8b4de24c86f5ca8a%7Eessss"},{"pid":"essss:38753","page":"122","label":"DSC09009","width":"2592","height":"1944","dsid":"JP2","token":"afd641ca0c8d8d7b132f1a8ee7af4b26ceb953ecb8f031b0abb8937995c4c8c2","identifier":"essss%3A38753%7EJP2%7Eafd641ca0c8d8d7b132f1a8ee7af4b26ceb953ecb8f031b0abb8937995c4c8c2%7Eessss"},{"pid":"essss:38754","page":"123","label":"DSC09010","width":"2592","height":"1944","dsid":"JP2","token":"d35d04f095629b53d9a743c11ce1651bcae304bb008dea648b46ae6fa5b750e2","identifier":"essss%3A38754%7EJP2%7Ed35d04f095629b53d9a743c11ce1651bcae304bb008dea648b46ae6fa5b750e2%7Eessss"},{"pid":"essss:38755","page":"124","label":"DSC09011","width":"2592","height":"1944","dsid":"JP2","token":"338dc4e819ce64bbe73f1c724cf0d82b90f1a394a8768e9451d68c75aa59b9ab","identifier":"essss%3A38755%7EJP2%7E338dc4e819ce64bbe73f1c724cf0d82b90f1a394a8768e9451d68c75aa59b9ab%7Eessss"},{"pid":"essss:38756","page":"125","label":"DSC09012","width":"2592","height":"1944","dsid":"JP2","token":"7c0b2f332298a9c8bb76487b46179982d13fd6427de8698fbeec08a64845bd48","identifier":"essss%3A38756%7EJP2%7E7c0b2f332298a9c8bb76487b46179982d13fd6427de8698fbeec08a64845bd48%7Eessss"},{"pid":"essss:38757","page":"126","label":"DSC09013","width":"2592","height":"1944","dsid":"JP2","token":"e5c23c7a64502cd848dfa6009ddf08190264c696b193cfb7381927bda43f0f3f","identifier":"essss%3A38757%7EJP2%7Ee5c23c7a64502cd848dfa6009ddf08190264c696b193cfb7381927bda43f0f3f%7Eessss"},{"pid":"essss:38758","page":"127","label":"DSC09014","width":"2592","height":"1944","dsid":"JP2","token":"5bb8be4eac6813dfa4d33169ca53901d23f5cc574217fcd684553749f17072c0","identifier":"essss%3A38758%7EJP2%7E5bb8be4eac6813dfa4d33169ca53901d23f5cc574217fcd684553749f17072c0%7Eessss"},{"pid":"essss:38759","page":"128","label":"DSC09015","width":"2592","height":"1944","dsid":"JP2","token":"9fc9e6a1ecf11921e05daaf4cc8a0b2258b24d5a037e776021c400ac25567c94","identifier":"essss%3A38759%7EJP2%7E9fc9e6a1ecf11921e05daaf4cc8a0b2258b24d5a037e776021c400ac25567c94%7Eessss"},{"pid":"essss:38760","page":"129","label":"DSC09016","width":"2592","height":"1944","dsid":"JP2","token":"fe48620fc2443948b5ff4580bcc62fd67e2a908fd10e1a56a507bf9e5a048fa6","identifier":"essss%3A38760%7EJP2%7Efe48620fc2443948b5ff4580bcc62fd67e2a908fd10e1a56a507bf9e5a048fa6%7Eessss"},{"pid":"essss:38761","page":"130","label":"DSC09017","width":"2592","height":"1944","dsid":"JP2","token":"9f81f20b5635e5d1ba4bd717992b907f921f4044dbf0fb57239dc65ece02aa7c","identifier":"essss%3A38761%7EJP2%7E9f81f20b5635e5d1ba4bd717992b907f921f4044dbf0fb57239dc65ece02aa7c%7Eessss"},{"pid":"essss:38762","page":"131","label":"DSC09018","width":"2592","height":"1944","dsid":"JP2","token":"a6398bceca6b4e498bc969d6b8e4a1eebded1da0f7a2d7bffbf509d37874f527","identifier":"essss%3A38762%7EJP2%7Ea6398bceca6b4e498bc969d6b8e4a1eebded1da0f7a2d7bffbf509d37874f527%7Eessss"},{"pid":"essss:38763","page":"132","label":"DSC09019","width":"2592","height":"1944","dsid":"JP2","token":"591efd9a51c16833a2048b0b2abbdbb1f86c5f7bd37bd09c5734ab6964a2da27","identifier":"essss%3A38763%7EJP2%7E591efd9a51c16833a2048b0b2abbdbb1f86c5f7bd37bd09c5734ab6964a2da27%7Eessss"},{"pid":"essss:38764","page":"133","label":"DSC09020","width":"2592","height":"1944","dsid":"JP2","token":"d252f27f4a799d95b4dba045563761e5bff197f2e552ee9fcf060bc0bd0119e3","identifier":"essss%3A38764%7EJP2%7Ed252f27f4a799d95b4dba045563761e5bff197f2e552ee9fcf060bc0bd0119e3%7Eessss"},{"pid":"essss:38765","page":"134","label":"DSC09021","width":"2592","height":"1944","dsid":"JP2","token":"20db478516f4179f2f3af7e5f87795a0b58520075cddc676583bd5c7424056c2","identifier":"essss%3A38765%7EJP2%7E20db478516f4179f2f3af7e5f87795a0b58520075cddc676583bd5c7424056c2%7Eessss"},{"pid":"essss:38766","page":"135","label":"DSC09022","width":"2592","height":"1944","dsid":"JP2","token":"08c5c08290177b585d832ec7b928da41f85798a686f42799023073cf20608ea4","identifier":"essss%3A38766%7EJP2%7E08c5c08290177b585d832ec7b928da41f85798a686f42799023073cf20608ea4%7Eessss"},{"pid":"essss:38767","page":"136","label":"DSC09023","width":"2592","height":"1944","dsid":"JP2","token":"3c623d03151e4d5f8a1039c4269eec815e904509f547cd498129a531b0f44139","identifier":"essss%3A38767%7EJP2%7E3c623d03151e4d5f8a1039c4269eec815e904509f547cd498129a531b0f44139%7Eessss"},{"pid":"essss:38768","page":"137","label":"DSC09024","width":"2592","height":"1944","dsid":"JP2","token":"cde8325e8790d8d162fc88bc7451ad94ebb36cb8af969f2de35db93b5ddff41a","identifier":"essss%3A38768%7EJP2%7Ecde8325e8790d8d162fc88bc7451ad94ebb36cb8af969f2de35db93b5ddff41a%7Eessss"},{"pid":"essss:38769","page":"138","label":"DSC09025","width":"2592","height":"1944","dsid":"JP2","token":"7d0abb357432c025c23fe1966690f9e65703eac94964a825e19328db8df7c914","identifier":"essss%3A38769%7EJP2%7E7d0abb357432c025c23fe1966690f9e65703eac94964a825e19328db8df7c914%7Eessss"},{"pid":"essss:38770","page":"139","label":"DSC09026","width":"2592","height":"1944","dsid":"JP2","token":"19947559789b019c10d8dc9e79060899b3de6ff8589d17bd5d17bb090cdf4eba","identifier":"essss%3A38770%7EJP2%7E19947559789b019c10d8dc9e79060899b3de6ff8589d17bd5d17bb090cdf4eba%7Eessss"},{"pid":"essss:38771","page":"140","label":"DSC09027","width":"2592","height":"1944","dsid":"JP2","token":"642794b426bafb973511f4db35cac9093a009edf56eb8bce6cf787c1457c49e4","identifier":"essss%3A38771%7EJP2%7E642794b426bafb973511f4db35cac9093a009edf56eb8bce6cf787c1457c49e4%7Eessss"},{"pid":"essss:38772","page":"141","label":"DSC09028","width":"2592","height":"1944","dsid":"JP2","token":"6d9558993b1ec2afc66697b09de10019931d25580ae3b49c19c3c586dc02bc5e","identifier":"essss%3A38772%7EJP2%7E6d9558993b1ec2afc66697b09de10019931d25580ae3b49c19c3c586dc02bc5e%7Eessss"},{"pid":"essss:38773","page":"142","label":"DSC09029","width":"2592","height":"1944","dsid":"JP2","token":"51c156d6524a2bd5fd50a398f28ee2293c4ac1f01f4bde7ece187407cc068a5b","identifier":"essss%3A38773%7EJP2%7E51c156d6524a2bd5fd50a398f28ee2293c4ac1f01f4bde7ece187407cc068a5b%7Eessss"},{"pid":"essss:38774","page":"143","label":"DSC09030","width":"2592","height":"1944","dsid":"JP2","token":"2e1a7965af6d7aac46cea3486e0096cc7b4b10204ad08b79fa4a422352d06b03","identifier":"essss%3A38774%7EJP2%7E2e1a7965af6d7aac46cea3486e0096cc7b4b10204ad08b79fa4a422352d06b03%7Eessss"},{"pid":"essss:38775","page":"144","label":"DSC09031","width":"2592","height":"1944","dsid":"JP2","token":"c361954fdc15e5f0331407d2e5db53bf5c4e3df568bf9b23f81690d764a916a7","identifier":"essss%3A38775%7EJP2%7Ec361954fdc15e5f0331407d2e5db53bf5c4e3df568bf9b23f81690d764a916a7%7Eessss"},{"pid":"essss:38776","page":"145","label":"DSC09032","width":"2592","height":"1944","dsid":"JP2","token":"01f8a7eb19033b3b9c344baa32b84f8d4e15dd4c684eb09423daa62f2c1f6c89","identifier":"essss%3A38776%7EJP2%7E01f8a7eb19033b3b9c344baa32b84f8d4e15dd4c684eb09423daa62f2c1f6c89%7Eessss"},{"pid":"essss:38777","page":"146","label":"DSC09033","width":"2592","height":"1944","dsid":"JP2","token":"e34252712766b9f843404f7806030684d5bca1ea79f68156ab2458ad840489af","identifier":"essss%3A38777%7EJP2%7Ee34252712766b9f843404f7806030684d5bca1ea79f68156ab2458ad840489af%7Eessss"},{"pid":"essss:38778","page":"147","label":"DSC09034","width":"2592","height":"1944","dsid":"JP2","token":"503107c785d063415e8ad0ff146d42363e5f761e867691b6490bbd8424e8cac6","identifier":"essss%3A38778%7EJP2%7E503107c785d063415e8ad0ff146d42363e5f761e867691b6490bbd8424e8cac6%7Eessss"},{"pid":"essss:38779","page":"148","label":"DSC09035","width":"2592","height":"1944","dsid":"JP2","token":"aab3478501410d8b15739112dc19629ff95ef3b4d1472d4cf8efbc2787d5281c","identifier":"essss%3A38779%7EJP2%7Eaab3478501410d8b15739112dc19629ff95ef3b4d1472d4cf8efbc2787d5281c%7Eessss"},{"pid":"essss:38780","page":"149","label":"DSC09036","width":"2592","height":"1944","dsid":"JP2","token":"26984dff44b1ebca79958d7a45b2dff928ff6a884110e8e11b6e4ae8d476bddc","identifier":"essss%3A38780%7EJP2%7E26984dff44b1ebca79958d7a45b2dff928ff6a884110e8e11b6e4ae8d476bddc%7Eessss"},{"pid":"essss:38781","page":"150","label":"DSC09037","width":"2592","height":"1944","dsid":"JP2","token":"29ba4d4c26071a15a571d535a95bcfc58ad1759c9c7caeeeff5dcd1ff73c215a","identifier":"essss%3A38781%7EJP2%7E29ba4d4c26071a15a571d535a95bcfc58ad1759c9c7caeeeff5dcd1ff73c215a%7Eessss"},{"pid":"essss:38782","page":"151","label":"DSC09038","width":"2592","height":"1944","dsid":"JP2","token":"66b048678d9079f22c91e1e231b2f79e4c9311dbd27b4d1f7f45546bb3dc1f27","identifier":"essss%3A38782%7EJP2%7E66b048678d9079f22c91e1e231b2f79e4c9311dbd27b4d1f7f45546bb3dc1f27%7Eessss"},{"pid":"essss:38783","page":"152","label":"DSC09039","width":"2592","height":"1944","dsid":"JP2","token":"55df7678e199099a9dc3acc636bacf37d5309a8482a24a6d14a1eddb0537a138","identifier":"essss%3A38783%7EJP2%7E55df7678e199099a9dc3acc636bacf37d5309a8482a24a6d14a1eddb0537a138%7Eessss"},{"pid":"essss:38784","page":"153","label":"DSC09040","width":"2592","height":"1944","dsid":"JP2","token":"8fe21d35022a82002aec774825aebe28d39a6d58a08399dc21b31586b8ebebfe","identifier":"essss%3A38784%7EJP2%7E8fe21d35022a82002aec774825aebe28d39a6d58a08399dc21b31586b8ebebfe%7Eessss"},{"pid":"essss:38785","page":"154","label":"DSC09041","width":"2592","height":"1944","dsid":"JP2","token":"acc4129c5b45e8b7b4d4935dbcfc1ee3b8722cf36aa9cb62a9671c6dc97fa00a","identifier":"essss%3A38785%7EJP2%7Eacc4129c5b45e8b7b4d4935dbcfc1ee3b8722cf36aa9cb62a9671c6dc97fa00a%7Eessss"},{"pid":"essss:38786","page":"155","label":"DSC09042","width":"2592","height":"1944","dsid":"JP2","token":"8234d271f6ed45334da616ed9283f0733f7da04421377bb9d954abb34c0c0bf9","identifier":"essss%3A38786%7EJP2%7E8234d271f6ed45334da616ed9283f0733f7da04421377bb9d954abb34c0c0bf9%7Eessss"},{"pid":"essss:38787","page":"156","label":"DSC09043","width":"2592","height":"1944","dsid":"JP2","token":"40e1bd436ca18a2954c0dd97aa60980b83be0aaa36779c57711de1003e620a4d","identifier":"essss%3A38787%7EJP2%7E40e1bd436ca18a2954c0dd97aa60980b83be0aaa36779c57711de1003e620a4d%7Eessss"},{"pid":"essss:38788","page":"157","label":"DSC09044","width":"2592","height":"1944","dsid":"JP2","token":"420478d98249f8d83c6a265fda26bbc88e2b8e71704cb35dd403fd7ad3ac9e29","identifier":"essss%3A38788%7EJP2%7E420478d98249f8d83c6a265fda26bbc88e2b8e71704cb35dd403fd7ad3ac9e29%7Eessss"},{"pid":"essss:38789","page":"158","label":"DSC09045","width":"2592","height":"1944","dsid":"JP2","token":"bb095d4ec7001589fa01723f9bdba1fdb613e7215fd3f5888a936bb1a94dc28b","identifier":"essss%3A38789%7EJP2%7Ebb095d4ec7001589fa01723f9bdba1fdb613e7215fd3f5888a936bb1a94dc28b%7Eessss"},{"pid":"essss:38790","page":"159","label":"DSC09046","width":"2592","height":"1944","dsid":"JP2","token":"5e80c1a134f3764324275aa2ed9bbb8da41cdea3f9b20dd7326e8a03262b333e","identifier":"essss%3A38790%7EJP2%7E5e80c1a134f3764324275aa2ed9bbb8da41cdea3f9b20dd7326e8a03262b333e%7Eessss"},{"pid":"essss:38791","page":"160","label":"DSC09047","width":"2592","height":"1944","dsid":"JP2","token":"270da910e0d1fe5e7e146025a333f79a167bb8f980c17747a1f4ecc621c8ddd9","identifier":"essss%3A38791%7EJP2%7E270da910e0d1fe5e7e146025a333f79a167bb8f980c17747a1f4ecc621c8ddd9%7Eessss"},{"pid":"essss:38792","page":"161","label":"DSC09048","width":"2592","height":"1944","dsid":"JP2","token":"94cc453168252502ea884486e82ce168d52955a402d80ba3989f263a28d88902","identifier":"essss%3A38792%7EJP2%7E94cc453168252502ea884486e82ce168d52955a402d80ba3989f263a28d88902%7Eessss"},{"pid":"essss:38793","page":"162","label":"DSC09049","width":"2592","height":"1944","dsid":"JP2","token":"722e78ee40913e1a326270b50e837e90eeef7ea9dccd232bdf3f2137d5b50bf5","identifier":"essss%3A38793%7EJP2%7E722e78ee40913e1a326270b50e837e90eeef7ea9dccd232bdf3f2137d5b50bf5%7Eessss"},{"pid":"essss:38794","page":"163","label":"DSC09050","width":"2592","height":"1944","dsid":"JP2","token":"a774cf43f87bf8d0ce6c24e7aed910dbc4d72b49242ed3bcb7c6f3edfaaa9c61","identifier":"essss%3A38794%7EJP2%7Ea774cf43f87bf8d0ce6c24e7aed910dbc4d72b49242ed3bcb7c6f3edfaaa9c61%7Eessss"},{"pid":"essss:38795","page":"164","label":"DSC09051","width":"2592","height":"1944","dsid":"JP2","token":"6ff048c20712c8306b1d27492c61e2477bb069bef9848f3b490b6b9c12ac6145","identifier":"essss%3A38795%7EJP2%7E6ff048c20712c8306b1d27492c61e2477bb069bef9848f3b490b6b9c12ac6145%7Eessss"},{"pid":"essss:38796","page":"165","label":"DSC09052","width":"2592","height":"1944","dsid":"JP2","token":"ee22c10edd906438caef806f81524247b47943d2abada087771f214d4e0e165a","identifier":"essss%3A38796%7EJP2%7Eee22c10edd906438caef806f81524247b47943d2abada087771f214d4e0e165a%7Eessss"},{"pid":"essss:38797","page":"166","label":"DSC09053","width":"2592","height":"1944","dsid":"JP2","token":"e2d56b757e2b2bd98e831cd8ba6fdd07c9ca2e89ffb0a9d03795870de0dc5590","identifier":"essss%3A38797%7EJP2%7Ee2d56b757e2b2bd98e831cd8ba6fdd07c9ca2e89ffb0a9d03795870de0dc5590%7Eessss"},{"pid":"essss:38798","page":"167","label":"DSC09054","width":"2592","height":"1944","dsid":"JP2","token":"112f2856bda05b0f6dd6555b2596b192532c9e8d425a640082a74894d778b2cc","identifier":"essss%3A38798%7EJP2%7E112f2856bda05b0f6dd6555b2596b192532c9e8d425a640082a74894d778b2cc%7Eessss"},{"pid":"essss:38799","page":"168","label":"DSC09055","width":"2592","height":"1944","dsid":"JP2","token":"3f60e379807f9ddeace0496b6e1e8cbe35782d0424cc677d2e2308f1e2aac92b","identifier":"essss%3A38799%7EJP2%7E3f60e379807f9ddeace0496b6e1e8cbe35782d0424cc677d2e2308f1e2aac92b%7Eessss"},{"pid":"essss:38800","page":"169","label":"DSC09056","width":"2592","height":"1944","dsid":"JP2","token":"6d94c5d8afaf3aa0e8e0e6dd7b4157699fe1cabfe914e27e70d1fbf6d8190c0e","identifier":"essss%3A38800%7EJP2%7E6d94c5d8afaf3aa0e8e0e6dd7b4157699fe1cabfe914e27e70d1fbf6d8190c0e%7Eessss"},{"pid":"essss:38801","page":"170","label":"DSC09057","width":"2592","height":"1944","dsid":"JP2","token":"c1032773d7127481019e1aaa415c6e5edc5be2e91ba48bf26122f71d11e9ff56","identifier":"essss%3A38801%7EJP2%7Ec1032773d7127481019e1aaa415c6e5edc5be2e91ba48bf26122f71d11e9ff56%7Eessss"},{"pid":"essss:38802","page":"171","label":"DSC09058","width":"2592","height":"1944","dsid":"JP2","token":"5b9916c6afd8c650b1c1c55baadc7579a6c0686983ba3c649d21a74499787b1d","identifier":"essss%3A38802%7EJP2%7E5b9916c6afd8c650b1c1c55baadc7579a6c0686983ba3c649d21a74499787b1d%7Eessss"},{"pid":"essss:38803","page":"172","label":"DSC09059","width":"2592","height":"1944","dsid":"JP2","token":"16dd2f879bdf7fbfab915c2e49c0cf9d819d755320ca34bfb6c85e0a14bc5470","identifier":"essss%3A38803%7EJP2%7E16dd2f879bdf7fbfab915c2e49c0cf9d819d755320ca34bfb6c85e0a14bc5470%7Eessss"},{"pid":"essss:38804","page":"173","label":"DSC09060","width":"2592","height":"1944","dsid":"JP2","token":"482be00941c66621aa7a5ee5c5230349af21a168ed2b726b65ebb875bc1467d6","identifier":"essss%3A38804%7EJP2%7E482be00941c66621aa7a5ee5c5230349af21a168ed2b726b65ebb875bc1467d6%7Eessss"},{"pid":"essss:38805","page":"174","label":"DSC09061","width":"2592","height":"1944","dsid":"JP2","token":"ff03d0d6bb56655ad7c4125c490d3b2341257a86a02368469000933f3eae94a3","identifier":"essss%3A38805%7EJP2%7Eff03d0d6bb56655ad7c4125c490d3b2341257a86a02368469000933f3eae94a3%7Eessss"},{"pid":"essss:38806","page":"175","label":"DSC09062","width":"2592","height":"1944","dsid":"JP2","token":"d3a395cadafabe146f9ad8ec53c8020c46c215d909f9df4e2581351646455f1d","identifier":"essss%3A38806%7EJP2%7Ed3a395cadafabe146f9ad8ec53c8020c46c215d909f9df4e2581351646455f1d%7Eessss"},{"pid":"essss:38807","page":"176","label":"DSC09063","width":"2592","height":"1944","dsid":"JP2","token":"8ee7e5464c4e1235cbbb8d3f16631b450a2c0209039e10d089f6ec753625f651","identifier":"essss%3A38807%7EJP2%7E8ee7e5464c4e1235cbbb8d3f16631b450a2c0209039e10d089f6ec753625f651%7Eessss"},{"pid":"essss:38808","page":"177","label":"DSC09064","width":"2592","height":"1944","dsid":"JP2","token":"eecc6ee0cc3f4b7ade300ca1d8199890712f9b2af2d403f47cdf1ea2d83415f1","identifier":"essss%3A38808%7EJP2%7Eeecc6ee0cc3f4b7ade300ca1d8199890712f9b2af2d403f47cdf1ea2d83415f1%7Eessss"},{"pid":"essss:38809","page":"178","label":"DSC09065","width":"2592","height":"1944","dsid":"JP2","token":"49516a1fdf5d922571a367afd59ea399f14ab2b15f3cc5471557518acbc8528f","identifier":"essss%3A38809%7EJP2%7E49516a1fdf5d922571a367afd59ea399f14ab2b15f3cc5471557518acbc8528f%7Eessss"},{"pid":"essss:38810","page":"179","label":"DSC09066","width":"2592","height":"1944","dsid":"JP2","token":"546737aa90b705c7cd56ec514109ba9be1460125af5484bc8123a45d1901a492","identifier":"essss%3A38810%7EJP2%7E546737aa90b705c7cd56ec514109ba9be1460125af5484bc8123a45d1901a492%7Eessss"},{"pid":"essss:38811","page":"180","label":"DSC09067","width":"2592","height":"1944","dsid":"JP2","token":"9aac1fccdeca261f8e98d28cc3cadb652f9e137fdb1adf0d00ed86d3071ba716","identifier":"essss%3A38811%7EJP2%7E9aac1fccdeca261f8e98d28cc3cadb652f9e137fdb1adf0d00ed86d3071ba716%7Eessss"},{"pid":"essss:38812","page":"181","label":"DSC09068","width":"2592","height":"1944","dsid":"JP2","token":"147f175f1da58f1ee73af30c93ff5a13fe85ecbffd7038c90a1d7f46e1c98605","identifier":"essss%3A38812%7EJP2%7E147f175f1da58f1ee73af30c93ff5a13fe85ecbffd7038c90a1d7f46e1c98605%7Eessss"},{"pid":"essss:38813","page":"182","label":"DSC09069","width":"2592","height":"1944","dsid":"JP2","token":"bd7a6a8a90c685bd45105d5f9afa8742f2c0174e352a1ec711c742edc812edcc","identifier":"essss%3A38813%7EJP2%7Ebd7a6a8a90c685bd45105d5f9afa8742f2c0174e352a1ec711c742edc812edcc%7Eessss"},{"pid":"essss:38814","page":"183","label":"DSC09070","width":"2592","height":"1944","dsid":"JP2","token":"1191de08b6cbd7b6d0acb6b9834615756dc7e06f9aad31cea3227ba6cf4c04f2","identifier":"essss%3A38814%7EJP2%7E1191de08b6cbd7b6d0acb6b9834615756dc7e06f9aad31cea3227ba6cf4c04f2%7Eessss"},{"pid":"essss:38815","page":"184","label":"DSC09071","width":"2592","height":"1944","dsid":"JP2","token":"30dd389507193042ffd3b9ceedcbdd5fda97ca0814cccfe03fadc073b700ea28","identifier":"essss%3A38815%7EJP2%7E30dd389507193042ffd3b9ceedcbdd5fda97ca0814cccfe03fadc073b700ea28%7Eessss"},{"pid":"essss:38816","page":"185","label":"DSC09072","width":"2592","height":"1944","dsid":"JP2","token":"32e8110f5111a2418b27fb12d377ecacf56c3d2f50f520dcf1e9c0d772c2e4b8","identifier":"essss%3A38816%7EJP2%7E32e8110f5111a2418b27fb12d377ecacf56c3d2f50f520dcf1e9c0d772c2e4b8%7Eessss"},{"pid":"essss:38817","page":"186","label":"DSC09073","width":"2592","height":"1944","dsid":"JP2","token":"c6f83f4b78e4b90e729b1c2053369b763241169897a29751cf98b7c78f6aa544","identifier":"essss%3A38817%7EJP2%7Ec6f83f4b78e4b90e729b1c2053369b763241169897a29751cf98b7c78f6aa544%7Eessss"},{"pid":"essss:38818","page":"187","label":"DSC09074","width":"2592","height":"1944","dsid":"JP2","token":"48adcafffde86aeda70413e6543535226679208d528c3fa296a636892bf5850d","identifier":"essss%3A38818%7EJP2%7E48adcafffde86aeda70413e6543535226679208d528c3fa296a636892bf5850d%7Eessss"},{"pid":"essss:38819","page":"188","label":"DSC09075","width":"2592","height":"1944","dsid":"JP2","token":"390f5d3afaa00d1f93f549d76dc80e89195c1f30c0f9fbc4e4ff5e2e7fc569c9","identifier":"essss%3A38819%7EJP2%7E390f5d3afaa00d1f93f549d76dc80e89195c1f30c0f9fbc4e4ff5e2e7fc569c9%7Eessss"},{"pid":"essss:38820","page":"189","label":"DSC09076","width":"2592","height":"1944","dsid":"JP2","token":"b53d5edbf7789d18f8ff3ff80be178cda99e4e715b5d74ce70c2fa1216fa7d45","identifier":"essss%3A38820%7EJP2%7Eb53d5edbf7789d18f8ff3ff80be178cda99e4e715b5d74ce70c2fa1216fa7d45%7Eessss"},{"pid":"essss:38821","page":"190","label":"DSC09077","width":"2592","height":"1944","dsid":"JP2","token":"930d5833befb81d561fae63c13a5ae8037598fa4b50c45272c039997c0690292","identifier":"essss%3A38821%7EJP2%7E930d5833befb81d561fae63c13a5ae8037598fa4b50c45272c039997c0690292%7Eessss"},{"pid":"essss:38822","page":"191","label":"DSC09078","width":"2592","height":"1944","dsid":"JP2","token":"372a4e77b3f98ecff7bffd0e1ecf3bbcfdd8c8578a47646aed2ac02a8b6c91ce","identifier":"essss%3A38822%7EJP2%7E372a4e77b3f98ecff7bffd0e1ecf3bbcfdd8c8578a47646aed2ac02a8b6c91ce%7Eessss"},{"pid":"essss:38823","page":"192","label":"DSC09079","width":"2592","height":"1944","dsid":"JP2","token":"5f17e7cc75d02482ebd9bd3663be5702954fbbd2b2d4dc51ed033995f541837f","identifier":"essss%3A38823%7EJP2%7E5f17e7cc75d02482ebd9bd3663be5702954fbbd2b2d4dc51ed033995f541837f%7Eessss"},{"pid":"essss:38824","page":"193","label":"DSC09080","width":"2592","height":"1944","dsid":"JP2","token":"939d5ddfdd14364163b0ab54bb504a4f60905ef5835bc13241df831fc0d91034","identifier":"essss%3A38824%7EJP2%7E939d5ddfdd14364163b0ab54bb504a4f60905ef5835bc13241df831fc0d91034%7Eessss"},{"pid":"essss:38825","page":"194","label":"DSC09081","width":"2592","height":"1944","dsid":"JP2","token":"4d4f21575fcc2faa028e17841bebc73838d9f2541c0200c840a82459ec1e00ce","identifier":"essss%3A38825%7EJP2%7E4d4f21575fcc2faa028e17841bebc73838d9f2541c0200c840a82459ec1e00ce%7Eessss"},{"pid":"essss:38826","page":"195","label":"DSC09082","width":"2592","height":"1944","dsid":"JP2","token":"66051c0433b64dba23ad682d4ebd95a936e6d28f22fbb6b803dd54d59311a931","identifier":"essss%3A38826%7EJP2%7E66051c0433b64dba23ad682d4ebd95a936e6d28f22fbb6b803dd54d59311a931%7Eessss"},{"pid":"essss:38827","page":"196","label":"DSC09083","width":"2592","height":"1944","dsid":"JP2","token":"a7e1f11f14ee5e37839e883384041f4b21fda8d9c14c5c309f333c5da6b1ef8c","identifier":"essss%3A38827%7EJP2%7Ea7e1f11f14ee5e37839e883384041f4b21fda8d9c14c5c309f333c5da6b1ef8c%7Eessss"},{"pid":"essss:38828","page":"197","label":"DSC09084","width":"2592","height":"1944","dsid":"JP2","token":"6e6c6d4b8747b0dfae2297f627693fe4b4444919669b8a98c6161bae39b87a06","identifier":"essss%3A38828%7EJP2%7E6e6c6d4b8747b0dfae2297f627693fe4b4444919669b8a98c6161bae39b87a06%7Eessss"},{"pid":"essss:38829","page":"198","label":"DSC09085","width":"2592","height":"1944","dsid":"JP2","token":"c59499761cc489652293d6866501593eb5c91e222fb44b0428defa8930f4fd89","identifier":"essss%3A38829%7EJP2%7Ec59499761cc489652293d6866501593eb5c91e222fb44b0428defa8930f4fd89%7Eessss"},{"pid":"essss:38830","page":"199","label":"DSC09086","width":"2592","height":"1944","dsid":"JP2","token":"673120619421d21b6a1af62a6be5acec76f0e330a5da160edae81bead0471a2a","identifier":"essss%3A38830%7EJP2%7E673120619421d21b6a1af62a6be5acec76f0e330a5da160edae81bead0471a2a%7Eessss"},{"pid":"essss:38831","page":"200","label":"DSC09087","width":"2592","height":"1944","dsid":"JP2","token":"901013e238169b89b9925bb96145b4609f000446049c9208d3463a4e095d1ac6","identifier":"essss%3A38831%7EJP2%7E901013e238169b89b9925bb96145b4609f000446049c9208d3463a4e095d1ac6%7Eessss"},{"pid":"essss:38832","page":"201","label":"DSC09088","width":"2592","height":"1944","dsid":"JP2","token":"5ae6f9df0abbb995652fd03583bea2cdf670bb1fbf08d036dee3566838c9f376","identifier":"essss%3A38832%7EJP2%7E5ae6f9df0abbb995652fd03583bea2cdf670bb1fbf08d036dee3566838c9f376%7Eessss"},{"pid":"essss:38833","page":"202","label":"DSC09089","width":"2592","height":"1944","dsid":"JP2","token":"b90d7bf635857fcef2ca059b38de720d559c8a056a1cfc14faefe0f3e5166e04","identifier":"essss%3A38833%7EJP2%7Eb90d7bf635857fcef2ca059b38de720d559c8a056a1cfc14faefe0f3e5166e04%7Eessss"},{"pid":"essss:38834","page":"203","label":"DSC09090","width":"2592","height":"1944","dsid":"JP2","token":"ea328a6391f12572d8f894fe3a5c7b328bce7654649a3ec0a36a395b04ea96f9","identifier":"essss%3A38834%7EJP2%7Eea328a6391f12572d8f894fe3a5c7b328bce7654649a3ec0a36a395b04ea96f9%7Eessss"},{"pid":"essss:38835","page":"204","label":"DSC09091","width":"2592","height":"1944","dsid":"JP2","token":"1457da587f133910d17c9faa34dad936b83f31e6786163fc5c49923706e09eda","identifier":"essss%3A38835%7EJP2%7E1457da587f133910d17c9faa34dad936b83f31e6786163fc5c49923706e09eda%7Eessss"},{"pid":"essss:38836","page":"205","label":"DSC09092","width":"2592","height":"1944","dsid":"JP2","token":"bbbb6a052e2d7e4ad852b31e9cb84c4f2a13dcd2c89bca24dc437b45676d2f7c","identifier":"essss%3A38836%7EJP2%7Ebbbb6a052e2d7e4ad852b31e9cb84c4f2a13dcd2c89bca24dc437b45676d2f7c%7Eessss"},{"pid":"essss:38837","page":"206","label":"DSC09093","width":"2592","height":"1944","dsid":"JP2","token":"7d1d0f875bcb3eb659722e92f79ae59f50180adcc5d569dd303f1f3378e8b7ae","identifier":"essss%3A38837%7EJP2%7E7d1d0f875bcb3eb659722e92f79ae59f50180adcc5d569dd303f1f3378e8b7ae%7Eessss"},{"pid":"essss:38838","page":"207","label":"DSC09094","width":"2592","height":"1944","dsid":"JP2","token":"db59be24ae0a943227fb59f21914a49dfb07694094af73bc310402341faeb65d","identifier":"essss%3A38838%7EJP2%7Edb59be24ae0a943227fb59f21914a49dfb07694094af73bc310402341faeb65d%7Eessss"},{"pid":"essss:38839","page":"208","label":"DSC09095","width":"2592","height":"1944","dsid":"JP2","token":"51ffcaf4176e2a085e8099e47eb47a86e306e64504d44a2721b4a976a4379861","identifier":"essss%3A38839%7EJP2%7E51ffcaf4176e2a085e8099e47eb47a86e306e64504d44a2721b4a976a4379861%7Eessss"},{"pid":"essss:38840","page":"209","label":"DSC09096","width":"2592","height":"1944","dsid":"JP2","token":"08af16b10917dd7b8e25ce2e6b88a9da334203eb96e53815b8773e13285e14b3","identifier":"essss%3A38840%7EJP2%7E08af16b10917dd7b8e25ce2e6b88a9da334203eb96e53815b8773e13285e14b3%7Eessss"},{"pid":"essss:38841","page":"210","label":"DSC09097","width":"2592","height":"1944","dsid":"JP2","token":"8c0072d25fb480dd6c572f1e22b80b7d39d0951621f89e6b54b75b64a73346c2","identifier":"essss%3A38841%7EJP2%7E8c0072d25fb480dd6c572f1e22b80b7d39d0951621f89e6b54b75b64a73346c2%7Eessss"},{"pid":"essss:38842","page":"211","label":"DSC09098","width":"2592","height":"1944","dsid":"JP2","token":"a49888fca675532d162d1dbeaf6eb82fa3835b806efabc7b456378380e3cc7c3","identifier":"essss%3A38842%7EJP2%7Ea49888fca675532d162d1dbeaf6eb82fa3835b806efabc7b456378380e3cc7c3%7Eessss"},{"pid":"essss:38843","page":"212","label":"DSC09099","width":"2592","height":"1944","dsid":"JP2","token":"c044f6c67f855d7787af0ae4f3d110682c3c6193035c3daa3c7bd651b6bd6b68","identifier":"essss%3A38843%7EJP2%7Ec044f6c67f855d7787af0ae4f3d110682c3c6193035c3daa3c7bd651b6bd6b68%7Eessss"},{"pid":"essss:38844","page":"213","label":"DSC09100","width":"2592","height":"1944","dsid":"JP2","token":"ad581fd21a31e86267ebc760145a88308abe12957fb5bb77a1b1da17fae4620d","identifier":"essss%3A38844%7EJP2%7Ead581fd21a31e86267ebc760145a88308abe12957fb5bb77a1b1da17fae4620d%7Eessss"},{"pid":"essss:38845","page":"214","label":"DSC09101","width":"2592","height":"1944","dsid":"JP2","token":"c69f99ae0fcf9915b45bd0524e7abd69caa3cf3a9ecd8c787ec8055f42676a7b","identifier":"essss%3A38845%7EJP2%7Ec69f99ae0fcf9915b45bd0524e7abd69caa3cf3a9ecd8c787ec8055f42676a7b%7Eessss"},{"pid":"essss:38846","page":"215","label":"DSC09102","width":"2592","height":"1944","dsid":"JP2","token":"155ac9a1369e42e989408fd3c3a3af5f5e64c630f8c45b6106b9d1de34a60146","identifier":"essss%3A38846%7EJP2%7E155ac9a1369e42e989408fd3c3a3af5f5e64c630f8c45b6106b9d1de34a60146%7Eessss"},{"pid":"essss:38847","page":"216","label":"DSC09103","width":"2592","height":"1944","dsid":"JP2","token":"5070d6c3f5607b190f14174a83d3823f37edbb2724bfe955d77a1294875b7550","identifier":"essss%3A38847%7EJP2%7E5070d6c3f5607b190f14174a83d3823f37edbb2724bfe955d77a1294875b7550%7Eessss"},{"pid":"essss:38848","page":"217","label":"DSC09104","width":"2592","height":"1944","dsid":"JP2","token":"8fc6d9285186590e7e9822947bb2af75f8e9c7be0382bedec9d5a1c0db07cf70","identifier":"essss%3A38848%7EJP2%7E8fc6d9285186590e7e9822947bb2af75f8e9c7be0382bedec9d5a1c0db07cf70%7Eessss"},{"pid":"essss:38849","page":"218","label":"DSC09105","width":"2592","height":"1944","dsid":"JP2","token":"db1e14a315374bb5eddfa010f18d218cf79090f4670fdefbba5437f7b3a5bfaf","identifier":"essss%3A38849%7EJP2%7Edb1e14a315374bb5eddfa010f18d218cf79090f4670fdefbba5437f7b3a5bfaf%7Eessss"},{"pid":"essss:38850","page":"219","label":"DSC09106","width":"2592","height":"1944","dsid":"JP2","token":"65e79c1ba7902974b56ffb606a489876d39c3b7d0a8c38a09f9e38be68faf287","identifier":"essss%3A38850%7EJP2%7E65e79c1ba7902974b56ffb606a489876d39c3b7d0a8c38a09f9e38be68faf287%7Eessss"},{"pid":"essss:38851","page":"220","label":"DSC09107","width":"2592","height":"1944","dsid":"JP2","token":"0b10ac07489586d4709f9e44848acac7a9d792e3fafd5313206bd71735d31091","identifier":"essss%3A38851%7EJP2%7E0b10ac07489586d4709f9e44848acac7a9d792e3fafd5313206bd71735d31091%7Eessss"},{"pid":"essss:38852","page":"221","label":"DSC09108","width":"2592","height":"1944","dsid":"JP2","token":"508941c09c74f2483928b80454fcf5c20d507321df2ac0a491aee4a16875ac34","identifier":"essss%3A38852%7EJP2%7E508941c09c74f2483928b80454fcf5c20d507321df2ac0a491aee4a16875ac34%7Eessss"},{"pid":"essss:38853","page":"222","label":"DSC09109","width":"2592","height":"1944","dsid":"JP2","token":"520202a620d3cc68aae2920dabc026c6fa9bfe331f54140227939b1376251983","identifier":"essss%3A38853%7EJP2%7E520202a620d3cc68aae2920dabc026c6fa9bfe331f54140227939b1376251983%7Eessss"},{"pid":"essss:38854","page":"223","label":"DSC09110","width":"2592","height":"1944","dsid":"JP2","token":"441286086ef3b8c82b63f433b6cb088a5e0df88bb72a9242de70c22ef9935644","identifier":"essss%3A38854%7EJP2%7E441286086ef3b8c82b63f433b6cb088a5e0df88bb72a9242de70c22ef9935644%7Eessss"},{"pid":"essss:38855","page":"224","label":"DSC09111","width":"2592","height":"1944","dsid":"JP2","token":"05cbdedd9b2365db056a35be88f2ba0f9d5c49b4961dbf5cef07c73dcc32631f","identifier":"essss%3A38855%7EJP2%7E05cbdedd9b2365db056a35be88f2ba0f9d5c49b4961dbf5cef07c73dcc32631f%7Eessss"},{"pid":"essss:38856","page":"225","label":"DSC09112","width":"2592","height":"1944","dsid":"JP2","token":"0ed9064b0d9a6de97210b0da3b8ca03216458763c1a8dc08b53e5b69e71a3063","identifier":"essss%3A38856%7EJP2%7E0ed9064b0d9a6de97210b0da3b8ca03216458763c1a8dc08b53e5b69e71a3063%7Eessss"},{"pid":"essss:38857","page":"226","label":"DSC09113","width":"2592","height":"1944","dsid":"JP2","token":"1942e9c076350ee571b65644978921471332c81cb7cae827ead61b7bdec9144f","identifier":"essss%3A38857%7EJP2%7E1942e9c076350ee571b65644978921471332c81cb7cae827ead61b7bdec9144f%7Eessss"},{"pid":"essss:38858","page":"227","label":"DSC09114","width":"2592","height":"1944","dsid":"JP2","token":"52b98a8e979aff67fd3e4efb7b0c7da9c695438e2fea24bb8144c60346c8c0a3","identifier":"essss%3A38858%7EJP2%7E52b98a8e979aff67fd3e4efb7b0c7da9c695438e2fea24bb8144c60346c8c0a3%7Eessss"},{"pid":"essss:38859","page":"228","label":"DSC09115","width":"2592","height":"1944","dsid":"JP2","token":"ac696df2ddf883c104a3a0dc4afcd41b786eee47671c1e3afb8b928aa79a8689","identifier":"essss%3A38859%7EJP2%7Eac696df2ddf883c104a3a0dc4afcd41b786eee47671c1e3afb8b928aa79a8689%7Eessss"},{"pid":"essss:38860","page":"229","label":"DSC09116","width":"2592","height":"1944","dsid":"JP2","token":"36d2c4072b65ef017ebc3830eb2f390353037daf9e34ac64bb1320ba88516370","identifier":"essss%3A38860%7EJP2%7E36d2c4072b65ef017ebc3830eb2f390353037daf9e34ac64bb1320ba88516370%7Eessss"},{"pid":"essss:38861","page":"230","label":"DSC09117","width":"2592","height":"1944","dsid":"JP2","token":"fca66b534aee61dfb3f0873fc5ec5140736108f3ea35c32c8a9217cd60888d8f","identifier":"essss%3A38861%7EJP2%7Efca66b534aee61dfb3f0873fc5ec5140736108f3ea35c32c8a9217cd60888d8f%7Eessss"},{"pid":"essss:38862","page":"231","label":"DSC09118","width":"2592","height":"1944","dsid":"JP2","token":"6ece831754e01eb85e4b270c2c6c4b1e67c0696c68bb70ca3af58b311806b066","identifier":"essss%3A38862%7EJP2%7E6ece831754e01eb85e4b270c2c6c4b1e67c0696c68bb70ca3af58b311806b066%7Eessss"},{"pid":"essss:38863","page":"232","label":"DSC09119","width":"2592","height":"1944","dsid":"JP2","token":"3620ca79dea4282cc0208bf3ef669f7dde9ffe1760410e286d824ca0df2ec288","identifier":"essss%3A38863%7EJP2%7E3620ca79dea4282cc0208bf3ef669f7dde9ffe1760410e286d824ca0df2ec288%7Eessss"},{"pid":"essss:38864","page":"233","label":"DSC09120","width":"2592","height":"1944","dsid":"JP2","token":"53d78e7031ec51c58afa6df808da1859b7ef2e77e502a94f01cdbf35f6e3d3ec","identifier":"essss%3A38864%7EJP2%7E53d78e7031ec51c58afa6df808da1859b7ef2e77e502a94f01cdbf35f6e3d3ec%7Eessss"},{"pid":"essss:38865","page":"234","label":"DSC09121","width":"2592","height":"1944","dsid":"JP2","token":"5271464f66e089a8ffb0af7b6d9de752acb6d9a756e75d95420c7e5421408468","identifier":"essss%3A38865%7EJP2%7E5271464f66e089a8ffb0af7b6d9de752acb6d9a756e75d95420c7e5421408468%7Eessss"},{"pid":"essss:38866","page":"235","label":"DSC09122","width":"2592","height":"1944","dsid":"JP2","token":"76892c2437f88dec0eb98bc0fc5c77c02110eecab67a999e41d545ac2c4ce7a2","identifier":"essss%3A38866%7EJP2%7E76892c2437f88dec0eb98bc0fc5c77c02110eecab67a999e41d545ac2c4ce7a2%7Eessss"},{"pid":"essss:38867","page":"236","label":"DSC09123","width":"2592","height":"1944","dsid":"JP2","token":"6bfd70cf3320296887cef8cf97ae3f708aeeb4141b46e3c5f804cf3345fddf57","identifier":"essss%3A38867%7EJP2%7E6bfd70cf3320296887cef8cf97ae3f708aeeb4141b46e3c5f804cf3345fddf57%7Eessss"},{"pid":"essss:38868","page":"237","label":"DSC09124","width":"2592","height":"1944","dsid":"JP2","token":"f521cb3fd5e05242400732df2e98d12074b1cdf0303074d3708d2621c1cb1ef3","identifier":"essss%3A38868%7EJP2%7Ef521cb3fd5e05242400732df2e98d12074b1cdf0303074d3708d2621c1cb1ef3%7Eessss"},{"pid":"essss:38869","page":"238","label":"DSC09125","width":"2592","height":"1944","dsid":"JP2","token":"dcac323d1761610c4a56483f9d46623069e6d974bf5c9c7c0eb24d6a1b74c00c","identifier":"essss%3A38869%7EJP2%7Edcac323d1761610c4a56483f9d46623069e6d974bf5c9c7c0eb24d6a1b74c00c%7Eessss"},{"pid":"essss:38870","page":"239","label":"DSC09126","width":"2592","height":"1944","dsid":"JP2","token":"645116f21ee700e7c7414db962135aee243caf73868597d43cc3bee052982e31","identifier":"essss%3A38870%7EJP2%7E645116f21ee700e7c7414db962135aee243caf73868597d43cc3bee052982e31%7Eessss"},{"pid":"essss:38871","page":"240","label":"DSC09127","width":"2592","height":"1944","dsid":"JP2","token":"8f380e1293a860700685dfd4614274203c21997b7b83cbf6da0c506ecb1f94b7","identifier":"essss%3A38871%7EJP2%7E8f380e1293a860700685dfd4614274203c21997b7b83cbf6da0c506ecb1f94b7%7Eessss"},{"pid":"essss:38872","page":"241","label":"DSC09128","width":"2592","height":"1944","dsid":"JP2","token":"decff87b94ce709a037dccab73d9b5ef52f4e86089a89a20749f781858dcbb04","identifier":"essss%3A38872%7EJP2%7Edecff87b94ce709a037dccab73d9b5ef52f4e86089a89a20749f781858dcbb04%7Eessss"},{"pid":"essss:38873","page":"242","label":"DSC09129","width":"2592","height":"1944","dsid":"JP2","token":"16e506be2d88c5c388ef8925a6abc5b0f40a0ebb179316ff0201e33b430d563b","identifier":"essss%3A38873%7EJP2%7E16e506be2d88c5c388ef8925a6abc5b0f40a0ebb179316ff0201e33b430d563b%7Eessss"},{"pid":"essss:38874","page":"243","label":"DSC09130","width":"2592","height":"1944","dsid":"JP2","token":"2f39484d71981bb476269e184e96237b63ac8f65d3fc47529897b6fb6f529f87","identifier":"essss%3A38874%7EJP2%7E2f39484d71981bb476269e184e96237b63ac8f65d3fc47529897b6fb6f529f87%7Eessss"},{"pid":"essss:38875","page":"244","label":"DSC09131","width":"2592","height":"1944","dsid":"JP2","token":"a16bf62f2b3f7b7683effadfbf5f44fe4b7f91f8e389a8a71fca98e260064a0f","identifier":"essss%3A38875%7EJP2%7Ea16bf62f2b3f7b7683effadfbf5f44fe4b7f91f8e389a8a71fca98e260064a0f%7Eessss"},{"pid":"essss:38876","page":"245","label":"DSC09132","width":"2592","height":"1944","dsid":"JP2","token":"6ba4c113f9fbdbdfa6f6871f4dfeaec715417f73dbd5c0858cbbcd472e1f0c36","identifier":"essss%3A38876%7EJP2%7E6ba4c113f9fbdbdfa6f6871f4dfeaec715417f73dbd5c0858cbbcd472e1f0c36%7Eessss"},{"pid":"essss:38877","page":"246","label":"DSC09133","width":"2592","height":"1944","dsid":"JP2","token":"a3de58793f178e96eef43616f865d474630e86ee2f10bfedb108ec2294a5b450","identifier":"essss%3A38877%7EJP2%7Ea3de58793f178e96eef43616f865d474630e86ee2f10bfedb108ec2294a5b450%7Eessss"},{"pid":"essss:38878","page":"247","label":"DSC09134","width":"2592","height":"1944","dsid":"JP2","token":"b8d389c22c71932c49771a14e68705a59b72d773d126d95a9009c54839589d60","identifier":"essss%3A38878%7EJP2%7Eb8d389c22c71932c49771a14e68705a59b72d773d126d95a9009c54839589d60%7Eessss"},{"pid":"essss:38879","page":"248","label":"DSC09135","width":"2592","height":"1944","dsid":"JP2","token":"79fdb4504f1aad53bc3e932d8e620c941cac5a1af6d9dd9b274470ed48b57b06","identifier":"essss%3A38879%7EJP2%7E79fdb4504f1aad53bc3e932d8e620c941cac5a1af6d9dd9b274470ed48b57b06%7Eessss"},{"pid":"essss:38880","page":"249","label":"DSC09136","width":"2592","height":"1944","dsid":"JP2","token":"9e950448d9f8566d46a952678a279dabaf779b27ee6ae107f88e635de1145fbc","identifier":"essss%3A38880%7EJP2%7E9e950448d9f8566d46a952678a279dabaf779b27ee6ae107f88e635de1145fbc%7Eessss"},{"pid":"essss:38881","page":"250","label":"DSC09137","width":"2592","height":"1944","dsid":"JP2","token":"11865fd6b62917fcfd60705b4b3f4028c352342fe2ac948901d77cce24142b87","identifier":"essss%3A38881%7EJP2%7E11865fd6b62917fcfd60705b4b3f4028c352342fe2ac948901d77cce24142b87%7Eessss"},{"pid":"essss:38882","page":"251","label":"DSC09138","width":"2592","height":"1944","dsid":"JP2","token":"8cdcb561696e2d0e3c6d5127e412403390f860342ebb01d0ff256e37b976d17c","identifier":"essss%3A38882%7EJP2%7E8cdcb561696e2d0e3c6d5127e412403390f860342ebb01d0ff256e37b976d17c%7Eessss"},{"pid":"essss:38883","page":"252","label":"DSC09139","width":"2592","height":"1944","dsid":"JP2","token":"1e3b15b02a9547f00ca9f7db8495da5322944f803312f0b4eea24b4cb2f1b9f2","identifier":"essss%3A38883%7EJP2%7E1e3b15b02a9547f00ca9f7db8495da5322944f803312f0b4eea24b4cb2f1b9f2%7Eessss"},{"pid":"essss:38884","page":"253","label":"DSC09140","width":"2592","height":"1944","dsid":"JP2","token":"02ce81fe455a932ec724bc028be5bf1f868c32d271b47d4c398d005b7d9ac66b","identifier":"essss%3A38884%7EJP2%7E02ce81fe455a932ec724bc028be5bf1f868c32d271b47d4c398d005b7d9ac66b%7Eessss"},{"pid":"essss:38885","page":"254","label":"DSC09141","width":"2592","height":"1944","dsid":"JP2","token":"fc22f0e68d5938aaf70eef27d8104dd5a1c3d3a7b3ed62be2d2c243c206de0c1","identifier":"essss%3A38885%7EJP2%7Efc22f0e68d5938aaf70eef27d8104dd5a1c3d3a7b3ed62be2d2c243c206de0c1%7Eessss"},{"pid":"essss:38886","page":"255","label":"DSC09142","width":"2592","height":"1944","dsid":"JP2","token":"3c45b4624216161123d62ba62bf4827b69f73038ad3c7f84b5cd1347ce371cb9","identifier":"essss%3A38886%7EJP2%7E3c45b4624216161123d62ba62bf4827b69f73038ad3c7f84b5cd1347ce371cb9%7Eessss"},{"pid":"essss:38887","page":"256","label":"DSC09143","width":"2592","height":"1944","dsid":"JP2","token":"a47c4d5d897f8f2ae0327f653cd1826d43a287be785231da19b7520a4eb89c53","identifier":"essss%3A38887%7EJP2%7Ea47c4d5d897f8f2ae0327f653cd1826d43a287be785231da19b7520a4eb89c53%7Eessss"},{"pid":"essss:38888","page":"257","label":"DSC09144","width":"2592","height":"1944","dsid":"JP2","token":"0c7c5f72aa3bad76417ca5eefd1b51ade55152c9fdc7bd51134be70eb83d5451","identifier":"essss%3A38888%7EJP2%7E0c7c5f72aa3bad76417ca5eefd1b51ade55152c9fdc7bd51134be70eb83d5451%7Eessss"},{"pid":"essss:38889","page":"258","label":"DSC09145","width":"2592","height":"1944","dsid":"JP2","token":"c431d5800912350c197957145846d5bcd3afec19af95fd55c4a996d2ea054503","identifier":"essss%3A38889%7EJP2%7Ec431d5800912350c197957145846d5bcd3afec19af95fd55c4a996d2ea054503%7Eessss"},{"pid":"essss:38890","page":"259","label":"DSC09146","width":"2592","height":"1944","dsid":"JP2","token":"d1b3cff4be2df898cdf29ec8b5393cbe995724ab15271915f5c8d5ab5983a16a","identifier":"essss%3A38890%7EJP2%7Ed1b3cff4be2df898cdf29ec8b5393cbe995724ab15271915f5c8d5ab5983a16a%7Eessss"},{"pid":"essss:38891","page":"260","label":"DSC09147","width":"2592","height":"1944","dsid":"JP2","token":"a2a1da254d55b7ee08ec4c3b0b90db7903461c69c3eb4f14fda298743a247ad6","identifier":"essss%3A38891%7EJP2%7Ea2a1da254d55b7ee08ec4c3b0b90db7903461c69c3eb4f14fda298743a247ad6%7Eessss"},{"pid":"essss:38892","page":"261","label":"DSC09148","width":"2592","height":"1944","dsid":"JP2","token":"63d99f853504bc7724ec191f29505263063f877a476a9ed1c1a8b199477c9a9c","identifier":"essss%3A38892%7EJP2%7E63d99f853504bc7724ec191f29505263063f877a476a9ed1c1a8b199477c9a9c%7Eessss"},{"pid":"essss:38893","page":"262","label":"DSC09149","width":"2592","height":"1944","dsid":"JP2","token":"a723cecf33fe65ead0c398e67ce981f0fbc0c8c54625aa42c70ab07eb7bb6738","identifier":"essss%3A38893%7EJP2%7Ea723cecf33fe65ead0c398e67ce981f0fbc0c8c54625aa42c70ab07eb7bb6738%7Eessss"},{"pid":"essss:38894","page":"263","label":"DSC09150","width":"2592","height":"1944","dsid":"JP2","token":"925382b8d1a635015ad390829f7f9c37b6a5ce68a01675035017d2cbb60793a2","identifier":"essss%3A38894%7EJP2%7E925382b8d1a635015ad390829f7f9c37b6a5ce68a01675035017d2cbb60793a2%7Eessss"},{"pid":"essss:38895","page":"264","label":"DSC09151","width":"2592","height":"1944","dsid":"JP2","token":"f06e7a32b3279c527ed632cd42b4c7e421656dc87c41e50c18460ddf8210f570","identifier":"essss%3A38895%7EJP2%7Ef06e7a32b3279c527ed632cd42b4c7e421656dc87c41e50c18460ddf8210f570%7Eessss"},{"pid":"essss:38896","page":"265","label":"DSC09152","width":"2592","height":"1944","dsid":"JP2","token":"facc9c029bb36b05c0513883fbcbe88ba3b5464bc186bcbd3b97c119eb2af354","identifier":"essss%3A38896%7EJP2%7Efacc9c029bb36b05c0513883fbcbe88ba3b5464bc186bcbd3b97c119eb2af354%7Eessss"},{"pid":"essss:38897","page":"266","label":"DSC09153","width":"2592","height":"1944","dsid":"JP2","token":"35c2e6e73e234ef2455a4d04d55bbecbfa61509ae00f62e0416c9d911b60f9f4","identifier":"essss%3A38897%7EJP2%7E35c2e6e73e234ef2455a4d04d55bbecbfa61509ae00f62e0416c9d911b60f9f4%7Eessss"},{"pid":"essss:38898","page":"267","label":"DSC09154","width":"2592","height":"1944","dsid":"JP2","token":"e7a652fde08913786b40dbf73ef44ea0dd24237078d1af9a542512740d8ff9e3","identifier":"essss%3A38898%7EJP2%7Ee7a652fde08913786b40dbf73ef44ea0dd24237078d1af9a542512740d8ff9e3%7Eessss"},{"pid":"essss:38899","page":"268","label":"DSC09155","width":"2592","height":"1944","dsid":"JP2","token":"9cd386745c96647a03897f58eb6e3648fb506802a4ee8a2531e27efd5b0018e2","identifier":"essss%3A38899%7EJP2%7E9cd386745c96647a03897f58eb6e3648fb506802a4ee8a2531e27efd5b0018e2%7Eessss"},{"pid":"essss:38900","page":"269","label":"DSC09156","width":"2592","height":"1944","dsid":"JP2","token":"d5ee1d7b61c9132a513aef498ca13a3927f7ab060528becac4483e8593d73837","identifier":"essss%3A38900%7EJP2%7Ed5ee1d7b61c9132a513aef498ca13a3927f7ab060528becac4483e8593d73837%7Eessss"},{"pid":"essss:38901","page":"270","label":"DSC09157","width":"2592","height":"1944","dsid":"JP2","token":"5c404579e0e8dec210622f8a99bdf6362a6c909f3105dd90e8fd78c2cbe57a2d","identifier":"essss%3A38901%7EJP2%7E5c404579e0e8dec210622f8a99bdf6362a6c909f3105dd90e8fd78c2cbe57a2d%7Eessss"},{"pid":"essss:38902","page":"271","label":"DSC09158","width":"2592","height":"1944","dsid":"JP2","token":"13f81c43f548d8ee8fb153bfa8eab27c5603bce654da616ebd2e6c3652bc30bd","identifier":"essss%3A38902%7EJP2%7E13f81c43f548d8ee8fb153bfa8eab27c5603bce654da616ebd2e6c3652bc30bd%7Eessss"},{"pid":"essss:38903","page":"272","label":"DSC09159","width":"2592","height":"1944","dsid":"JP2","token":"9f65378336636604a92f9d934039968017bf14f66bf7237169d4c13cd29acdaa","identifier":"essss%3A38903%7EJP2%7E9f65378336636604a92f9d934039968017bf14f66bf7237169d4c13cd29acdaa%7Eessss"},{"pid":"essss:38904","page":"273","label":"DSC09160","width":"2592","height":"1944","dsid":"JP2","token":"8beca25b7782aa03a655a54f247fdb7cad0ebeabc5b5185319fd4e653388f1b1","identifier":"essss%3A38904%7EJP2%7E8beca25b7782aa03a655a54f247fdb7cad0ebeabc5b5185319fd4e653388f1b1%7Eessss"},{"pid":"essss:38905","page":"274","label":"DSC09161","width":"2592","height":"1944","dsid":"JP2","token":"a7c51453fb60fd6bc4fb2cd5b8afe3053b42afec17479163a7b3354ca8437f9e","identifier":"essss%3A38905%7EJP2%7Ea7c51453fb60fd6bc4fb2cd5b8afe3053b42afec17479163a7b3354ca8437f9e%7Eessss"},{"pid":"essss:38906","page":"275","label":"DSC09162","width":"2592","height":"1944","dsid":"JP2","token":"40157aa6932b69d7cd22d83d9138021b9a4d5a54fd2ff52f83ca2b55cc694d72","identifier":"essss%3A38906%7EJP2%7E40157aa6932b69d7cd22d83d9138021b9a4d5a54fd2ff52f83ca2b55cc694d72%7Eessss"},{"pid":"essss:38907","page":"276","label":"DSC09163","width":"2592","height":"1944","dsid":"JP2","token":"c4bc398ece9f7d2d952e10ea976e0a706223b945a34aa60c4c1d29a488940fa8","identifier":"essss%3A38907%7EJP2%7Ec4bc398ece9f7d2d952e10ea976e0a706223b945a34aa60c4c1d29a488940fa8%7Eessss"},{"pid":"essss:38908","page":"277","label":"DSC09164","width":"2592","height":"1944","dsid":"JP2","token":"94c9e75334e650ccdfdbccc545e7bb808983f4f01f9ecde4bf8a54aa16d9d412","identifier":"essss%3A38908%7EJP2%7E94c9e75334e650ccdfdbccc545e7bb808983f4f01f9ecde4bf8a54aa16d9d412%7Eessss"},{"pid":"essss:38909","page":"278","label":"DSC09165","width":"2592","height":"1944","dsid":"JP2","token":"6087bf8c3e3d85cf1139c900037203de0450eaf076529b8e690ff331b8b34c3b","identifier":"essss%3A38909%7EJP2%7E6087bf8c3e3d85cf1139c900037203de0450eaf076529b8e690ff331b8b34c3b%7Eessss"},{"pid":"essss:38910","page":"279","label":"DSC09166","width":"2592","height":"1944","dsid":"JP2","token":"30f938be37ea99bd6680c452d2e20684e85944ffd864eb1e3cbf0162c85b635b","identifier":"essss%3A38910%7EJP2%7E30f938be37ea99bd6680c452d2e20684e85944ffd864eb1e3cbf0162c85b635b%7Eessss"},{"pid":"essss:38911","page":"280","label":"DSC09167","width":"2592","height":"1944","dsid":"JP2","token":"b81670aadfefede9dec9f0b3f4877c960bb44a3992899acb62294220ae3c62f9","identifier":"essss%3A38911%7EJP2%7Eb81670aadfefede9dec9f0b3f4877c960bb44a3992899acb62294220ae3c62f9%7Eessss"},{"pid":"essss:38912","page":"281","label":"DSC09168","width":"2592","height":"1944","dsid":"JP2","token":"51b98f031202a249c86831962288125c0493c291f4a48ed77f18dff5332f119d","identifier":"essss%3A38912%7EJP2%7E51b98f031202a249c86831962288125c0493c291f4a48ed77f18dff5332f119d%7Eessss"},{"pid":"essss:38913","page":"282","label":"DSC09169","width":"2592","height":"1944","dsid":"JP2","token":"e0b3026b9393fd703b14b78536c9217311de68e32622cf9bf5524da4500ca8f8","identifier":"essss%3A38913%7EJP2%7Ee0b3026b9393fd703b14b78536c9217311de68e32622cf9bf5524da4500ca8f8%7Eessss"},{"pid":"essss:38914","page":"283","label":"DSC09170","width":"2592","height":"1944","dsid":"JP2","token":"81e6bcc2563d4265e6d0f7d8bd9c40fa7a90c776e21893ad54a75aebd4a241ec","identifier":"essss%3A38914%7EJP2%7E81e6bcc2563d4265e6d0f7d8bd9c40fa7a90c776e21893ad54a75aebd4a241ec%7Eessss"},{"pid":"essss:38915","page":"284","label":"DSC09171","width":"2592","height":"1944","dsid":"JP2","token":"283ff56d9a2909d79900269b6b459fc8a9db0a78dc18ea1a639114d3ca5e4598","identifier":"essss%3A38915%7EJP2%7E283ff56d9a2909d79900269b6b459fc8a9db0a78dc18ea1a639114d3ca5e4598%7Eessss"},{"pid":"essss:38916","page":"285","label":"DSC09172","width":"2592","height":"1944","dsid":"JP2","token":"a1274b7ee18c732d5f5fc21a4db49ccafad9c0ca89f3b2606d7eed58fe4f7aad","identifier":"essss%3A38916%7EJP2%7Ea1274b7ee18c732d5f5fc21a4db49ccafad9c0ca89f3b2606d7eed58fe4f7aad%7Eessss"},{"pid":"essss:38917","page":"286","label":"DSC09173","width":"2592","height":"1944","dsid":"JP2","token":"c4049fde10f4e82f15bb39efe99577ee9e5cfdbce43d4d10ae902762331f6f0a","identifier":"essss%3A38917%7EJP2%7Ec4049fde10f4e82f15bb39efe99577ee9e5cfdbce43d4d10ae902762331f6f0a%7Eessss"},{"pid":"essss:38918","page":"287","label":"DSC09174","width":"2592","height":"1944","dsid":"JP2","token":"6cac03a5050d0af7ba02b863e762b499d6c21faf3db6334352ad665eb932c0d4","identifier":"essss%3A38918%7EJP2%7E6cac03a5050d0af7ba02b863e762b499d6c21faf3db6334352ad665eb932c0d4%7Eessss"},{"pid":"essss:38919","page":"288","label":"DSC09175","width":"2592","height":"1944","dsid":"JP2","token":"e598bdfe6f097e1f80042d9f2d70fcc5a0588a3a1e2a2e455f89c8e685b76e84","identifier":"essss%3A38919%7EJP2%7Ee598bdfe6f097e1f80042d9f2d70fcc5a0588a3a1e2a2e455f89c8e685b76e84%7Eessss"},{"pid":"essss:38920","page":"289","label":"DSC09176","width":"2592","height":"1944","dsid":"JP2","token":"c7533718ad99ee41dfb393116b47f10d8133e75be0d4f15f5e0e1a1722d7cb31","identifier":"essss%3A38920%7EJP2%7Ec7533718ad99ee41dfb393116b47f10d8133e75be0d4f15f5e0e1a1722d7cb31%7Eessss"},{"pid":"essss:38921","page":"290","label":"DSC09177","width":"2592","height":"1944","dsid":"JP2","token":"dabd37e9d20535603058f8eec23106e22b268fbf09b81d9ae02902f456168dc7","identifier":"essss%3A38921%7EJP2%7Edabd37e9d20535603058f8eec23106e22b268fbf09b81d9ae02902f456168dc7%7Eessss"},{"pid":"essss:38922","page":"291","label":"DSC09178","width":"2592","height":"1944","dsid":"JP2","token":"cf222bdf30a23f9c14d603b2631d002fb91c4871e718ca6630449619e0288070","identifier":"essss%3A38922%7EJP2%7Ecf222bdf30a23f9c14d603b2631d002fb91c4871e718ca6630449619e0288070%7Eessss"},{"pid":"essss:38923","page":"292","label":"DSC09179","width":"2592","height":"1944","dsid":"JP2","token":"1b3506ed294b5141c903843b39cb611a6a93ecfe772ede1b87ae1bbebd299648","identifier":"essss%3A38923%7EJP2%7E1b3506ed294b5141c903843b39cb611a6a93ecfe772ede1b87ae1bbebd299648%7Eessss"},{"pid":"essss:38924","page":"293","label":"DSC09180","width":"2592","height":"1944","dsid":"JP2","token":"d21eb24d73f178c0d993887497aa11fe9835d1ae112914bac15d746083f04484","identifier":"essss%3A38924%7EJP2%7Ed21eb24d73f178c0d993887497aa11fe9835d1ae112914bac15d746083f04484%7Eessss"},{"pid":"essss:38925","page":"294","label":"DSC09181","width":"2592","height":"1944","dsid":"JP2","token":"ca262d2a0716f354a629e243582e6c4f862c7aaff9d73c177b1570018071629e","identifier":"essss%3A38925%7EJP2%7Eca262d2a0716f354a629e243582e6c4f862c7aaff9d73c177b1570018071629e%7Eessss"},{"pid":"essss:38926","page":"295","label":"DSC09182","width":"2592","height":"1944","dsid":"JP2","token":"f81f2998bc46d37910e245e660b0ead84a48aa12f114b43c4bc28fb29b7256cb","identifier":"essss%3A38926%7EJP2%7Ef81f2998bc46d37910e245e660b0ead84a48aa12f114b43c4bc28fb29b7256cb%7Eessss"},{"pid":"essss:38927","page":"296","label":"DSC09183","width":"2592","height":"1944","dsid":"JP2","token":"40ba212d16b53ea73734d4beed8ad9811ff4890b825d8ab84279a3c97265845f","identifier":"essss%3A38927%7EJP2%7E40ba212d16b53ea73734d4beed8ad9811ff4890b825d8ab84279a3c97265845f%7Eessss"},{"pid":"essss:38928","page":"297","label":"DSC09184","width":"2592","height":"1944","dsid":"JP2","token":"e447d066286ae43b9a7930c66529ef54fc13894440a8b0fbc3a240cd02a56b5b","identifier":"essss%3A38928%7EJP2%7Ee447d066286ae43b9a7930c66529ef54fc13894440a8b0fbc3a240cd02a56b5b%7Eessss"},{"pid":"essss:38929","page":"298","label":"DSC09185","width":"2592","height":"1944","dsid":"JP2","token":"2c01e093ae3da15f870a071cad981d242441db82b50a0e848b5ec06e5d3bbb98","identifier":"essss%3A38929%7EJP2%7E2c01e093ae3da15f870a071cad981d242441db82b50a0e848b5ec06e5d3bbb98%7Eessss"},{"pid":"essss:38930","page":"299","label":"DSC09186","width":"2592","height":"1944","dsid":"JP2","token":"ea2c282bcc4d168c129193a54dfad0bdacf6c1ccea68164b780ccf7adfeccdcf","identifier":"essss%3A38930%7EJP2%7Eea2c282bcc4d168c129193a54dfad0bdacf6c1ccea68164b780ccf7adfeccdcf%7Eessss"},{"pid":"essss:38931","page":"300","label":"DSC09187","width":"2592","height":"1944","dsid":"JP2","token":"2286e2616b1c91458c8302c1d4358fe5e725b5bf0f2a1eec534f08faa63c66a8","identifier":"essss%3A38931%7EJP2%7E2286e2616b1c91458c8302c1d4358fe5e725b5bf0f2a1eec534f08faa63c66a8%7Eessss"},{"pid":"essss:38932","page":"301","label":"DSC09188","width":"2592","height":"1944","dsid":"JP2","token":"3b8f1d57142dd6be3ec4bbdc36b406aa42022d9f6b9eb02f8f2e185b9acabb67","identifier":"essss%3A38932%7EJP2%7E3b8f1d57142dd6be3ec4bbdc36b406aa42022d9f6b9eb02f8f2e185b9acabb67%7Eessss"},{"pid":"essss:38933","page":"302","label":"DSC09189","width":"2592","height":"1944","dsid":"JP2","token":"16a8662d1939f2e91cd6251b9844d4ee51b1494d5935044833c521132f1b0862","identifier":"essss%3A38933%7EJP2%7E16a8662d1939f2e91cd6251b9844d4ee51b1494d5935044833c521132f1b0862%7Eessss"},{"pid":"essss:38934","page":"303","label":"DSC09190","width":"2592","height":"1944","dsid":"JP2","token":"db6c289acc17d69fe9a293c973004b1fe782e9adf6c39cab48c0b963b2a64a31","identifier":"essss%3A38934%7EJP2%7Edb6c289acc17d69fe9a293c973004b1fe782e9adf6c39cab48c0b963b2a64a31%7Eessss"},{"pid":"essss:38935","page":"304","label":"DSC09191","width":"2592","height":"1944","dsid":"JP2","token":"8f3e0242a2dced4ba158fe6282b1cb7a8ed55064abf304ad26b2f5f6e0fcd851","identifier":"essss%3A38935%7EJP2%7E8f3e0242a2dced4ba158fe6282b1cb7a8ed55064abf304ad26b2f5f6e0fcd851%7Eessss"},{"pid":"essss:38936","page":"305","label":"DSC09192","width":"2592","height":"1944","dsid":"JP2","token":"730523ffb284c1da8fd70a34a5cf7fd85288fa542158b833fe605eea017e7d82","identifier":"essss%3A38936%7EJP2%7E730523ffb284c1da8fd70a34a5cf7fd85288fa542158b833fe605eea017e7d82%7Eessss"},{"pid":"essss:38937","page":"306","label":"DSC09193","width":"2592","height":"1944","dsid":"JP2","token":"b39fb81cd2ec2d678f7ea5917953aa55220e689667944210d7d153f42b857940","identifier":"essss%3A38937%7EJP2%7Eb39fb81cd2ec2d678f7ea5917953aa55220e689667944210d7d153f42b857940%7Eessss"},{"pid":"essss:38938","page":"307","label":"DSC09194","width":"2592","height":"1944","dsid":"JP2","token":"7ba9bdf1a25a168e49a57d37f2fb759aa64a5cb3ce6114019c2f03cfc016a7c7","identifier":"essss%3A38938%7EJP2%7E7ba9bdf1a25a168e49a57d37f2fb759aa64a5cb3ce6114019c2f03cfc016a7c7%7Eessss"},{"pid":"essss:38939","page":"308","label":"DSC09195","width":"2592","height":"1944","dsid":"JP2","token":"8d6e02de5261a50f2205241cf1c7230c59e17ff0508b0798ed74cb3915b32ebf","identifier":"essss%3A38939%7EJP2%7E8d6e02de5261a50f2205241cf1c7230c59e17ff0508b0798ed74cb3915b32ebf%7Eessss"},{"pid":"essss:38940","page":"309","label":"DSC09196","width":"2592","height":"1944","dsid":"JP2","token":"9ae157366627d22ab1081c4e201418ecaf0a9bd3edab715f005fb097d6952a8b","identifier":"essss%3A38940%7EJP2%7E9ae157366627d22ab1081c4e201418ecaf0a9bd3edab715f005fb097d6952a8b%7Eessss"},{"pid":"essss:38941","page":"310","label":"DSC09197","width":"2592","height":"1944","dsid":"JP2","token":"119ebd0ad14ef14373ac4867c2e0e70037abf9e188f59e8d871b30c5eaf551d0","identifier":"essss%3A38941%7EJP2%7E119ebd0ad14ef14373ac4867c2e0e70037abf9e188f59e8d871b30c5eaf551d0%7Eessss"},{"pid":"essss:38942","page":"311","label":"DSC09198","width":"2592","height":"1944","dsid":"JP2","token":"841e02682131bedd123286d20a6d3f9aa34dfd3e577d695620e4b7e8050b11ce","identifier":"essss%3A38942%7EJP2%7E841e02682131bedd123286d20a6d3f9aa34dfd3e577d695620e4b7e8050b11ce%7Eessss"},{"pid":"essss:38943","page":"312","label":"DSC09199","width":"2592","height":"1944","dsid":"JP2","token":"57ea73bf1b5a0b9ea66a542a25ad4d770b79db97009e6ac73fc64cb8c2a7cf59","identifier":"essss%3A38943%7EJP2%7E57ea73bf1b5a0b9ea66a542a25ad4d770b79db97009e6ac73fc64cb8c2a7cf59%7Eessss"},{"pid":"essss:38944","page":"313","label":"DSC09200","width":"2592","height":"1944","dsid":"JP2","token":"c22c8b6a319e279944f2f793f780dfd226c2e9a2ed6ca6afe5fae41e94a43ffc","identifier":"essss%3A38944%7EJP2%7Ec22c8b6a319e279944f2f793f780dfd226c2e9a2ed6ca6afe5fae41e94a43ffc%7Eessss"},{"pid":"essss:38945","page":"314","label":"DSC09201","width":"2592","height":"1944","dsid":"JP2","token":"f223eae4a4dca24b63ed55ff4140d5d61d5d604e77d2d4dc186034e9a9691ebc","identifier":"essss%3A38945%7EJP2%7Ef223eae4a4dca24b63ed55ff4140d5d61d5d604e77d2d4dc186034e9a9691ebc%7Eessss"},{"pid":"essss:38946","page":"315","label":"DSC09202","width":"2592","height":"1944","dsid":"JP2","token":"c9baec69c64a59a766e3e46eb5d8e8fd7880a0cd8199b64999a7aaf00f1f0edd","identifier":"essss%3A38946%7EJP2%7Ec9baec69c64a59a766e3e46eb5d8e8fd7880a0cd8199b64999a7aaf00f1f0edd%7Eessss"},{"pid":"essss:38947","page":"316","label":"DSC09203","width":"2592","height":"1944","dsid":"JP2","token":"0587defb47ca59caf960aa18f54c9bf27ac3e7488855bc315df7527138e2ea73","identifier":"essss%3A38947%7EJP2%7E0587defb47ca59caf960aa18f54c9bf27ac3e7488855bc315df7527138e2ea73%7Eessss"},{"pid":"essss:38948","page":"317","label":"DSC09204","width":"2592","height":"1944","dsid":"JP2","token":"614f0a0cf3f72c136230e216a66bea5b8dabd0cd52c876949f7882985256c823","identifier":"essss%3A38948%7EJP2%7E614f0a0cf3f72c136230e216a66bea5b8dabd0cd52c876949f7882985256c823%7Eessss"},{"pid":"essss:38949","page":"318","label":"DSC09205","width":"2592","height":"1944","dsid":"JP2","token":"74b4b4f9398e60188b680d4c3ebd249d4a78c18f76c76bae5a184cff68613bb4","identifier":"essss%3A38949%7EJP2%7E74b4b4f9398e60188b680d4c3ebd249d4a78c18f76c76bae5a184cff68613bb4%7Eessss"},{"pid":"essss:38950","page":"319","label":"DSC09206","width":"2592","height":"1944","dsid":"JP2","token":"437e585b9a5e661d3b937c4d6f54b763b5a921902dddd65e0f0ac277a0c53aa1","identifier":"essss%3A38950%7EJP2%7E437e585b9a5e661d3b937c4d6f54b763b5a921902dddd65e0f0ac277a0c53aa1%7Eessss"},{"pid":"essss:38951","page":"320","label":"DSC09207","width":"2592","height":"1944","dsid":"JP2","token":"81371a20b6d476ee436febe609fffc89b9c426f7d6e0952db67bb799ecb0b656","identifier":"essss%3A38951%7EJP2%7E81371a20b6d476ee436febe609fffc89b9c426f7d6e0952db67bb799ecb0b656%7Eessss"},{"pid":"essss:38952","page":"321","label":"DSC09208","width":"2592","height":"1944","dsid":"JP2","token":"25caeef0c3062a077120793508c12d817230dbb762f6953650093c21e65590e1","identifier":"essss%3A38952%7EJP2%7E25caeef0c3062a077120793508c12d817230dbb762f6953650093c21e65590e1%7Eessss"},{"pid":"essss:38953","page":"322","label":"DSC09209","width":"2592","height":"1944","dsid":"JP2","token":"d10a43c33ca4fb85327a74ade55b6dcf6fdb07525ae619026e55f68ec0281dd2","identifier":"essss%3A38953%7EJP2%7Ed10a43c33ca4fb85327a74ade55b6dcf6fdb07525ae619026e55f68ec0281dd2%7Eessss"},{"pid":"essss:38954","page":"323","label":"DSC09210","width":"2592","height":"1944","dsid":"JP2","token":"fbdcc28bcb3839421a695d187244a12e50356c0297e413e2eaa4d1058981bca2","identifier":"essss%3A38954%7EJP2%7Efbdcc28bcb3839421a695d187244a12e50356c0297e413e2eaa4d1058981bca2%7Eessss"},{"pid":"essss:38955","page":"324","label":"DSC09211","width":"2592","height":"1944","dsid":"JP2","token":"4d4a01d0c87918ff5b9e060a859e76fb2977859d9b7e94f7077d52c4a35c0a14","identifier":"essss%3A38955%7EJP2%7E4d4a01d0c87918ff5b9e060a859e76fb2977859d9b7e94f7077d52c4a35c0a14%7Eessss"},{"pid":"essss:38956","page":"325","label":"DSC09212","width":"2592","height":"1944","dsid":"JP2","token":"023b0f612aa889e800421c3b3c224173fccf3ad7ff192d10b2f947f056e51341","identifier":"essss%3A38956%7EJP2%7E023b0f612aa889e800421c3b3c224173fccf3ad7ff192d10b2f947f056e51341%7Eessss"},{"pid":"essss:38957","page":"326","label":"DSC09213","width":"2592","height":"1944","dsid":"JP2","token":"856f92be2cb9dfea748499da19182c079a346f424c35d2538120005e985870d5","identifier":"essss%3A38957%7EJP2%7E856f92be2cb9dfea748499da19182c079a346f424c35d2538120005e985870d5%7Eessss"},{"pid":"essss:38958","page":"327","label":"DSC09214","width":"2592","height":"1944","dsid":"JP2","token":"6dd9a2ae5fd01024b32df408c9890ca3d051a8ee12302f7fdb25c64fddf2f939","identifier":"essss%3A38958%7EJP2%7E6dd9a2ae5fd01024b32df408c9890ca3d051a8ee12302f7fdb25c64fddf2f939%7Eessss"},{"pid":"essss:38959","page":"328","label":"DSC09215","width":"2592","height":"1944","dsid":"JP2","token":"ad8a36062e1ef5b9532f9cd2e21fe1f7def5249a45ca673de4cee9ddbe3dded5","identifier":"essss%3A38959%7EJP2%7Ead8a36062e1ef5b9532f9cd2e21fe1f7def5249a45ca673de4cee9ddbe3dded5%7Eessss"},{"pid":"essss:38960","page":"329","label":"DSC09216","width":"2592","height":"1944","dsid":"JP2","token":"1bfb53f631f13685e673be52a5bbcf3283b7c03fd15dffba9cd1882dbc98256f","identifier":"essss%3A38960%7EJP2%7E1bfb53f631f13685e673be52a5bbcf3283b7c03fd15dffba9cd1882dbc98256f%7Eessss"},{"pid":"essss:38961","page":"330","label":"DSC09217","width":"2592","height":"1944","dsid":"JP2","token":"719d62dfa5bf007850e22cfe10d811ca6414cc2c66c05f1f439708dbec353b4e","identifier":"essss%3A38961%7EJP2%7E719d62dfa5bf007850e22cfe10d811ca6414cc2c66c05f1f439708dbec353b4e%7Eessss"}]'

In [5]:
# rawjson from view-source:https://essss.library.vanderbilt.edu/islandora/object/essss%3A38558#page/1/mode/2up

In [6]:
urlpre = "https://essss.library.vanderbilt.edu/iiif/2/"
urlpost = "/full/pct:100/0/default.jpg"
# startpage = "https://essss.library.vanderbilt.edu/islandora/object/essss%3A38558#page/1/mode/2up"

In [7]:
headers = {
    "Accept": "image/avif,image/webp,image/apng,image/*,*/*;q=0.8",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en;q=0.9",
    "Connection": "keep-alive",
    "Cookie": "has_js=1; _ga=GA1.2.1752803547.1599356991; _gid=GA1.2.880756344.1599356991; _gat=1",
    "Host": "essss.library.vanderbilt.edu",
    "Referer": "https://essss.library.vanderbilt.edu/islandora/object/essss%3A38558",
    "Sec-Fetch-Dest": "image",
    "Sec-Fetch-Mode": "no-cors",
    "Sec-Fetch-Site": "same-origin"
}

In [8]:
for startpage in urlswanted:
    r = requests.get(startpage)
    html = r.content
    bookname = pq(html)("li.active")[0].text
    bookdir = "books/" + bookname + "/"
    rawdir = bookdir + "raw/"
    processeddir = bookdir + "processed/"
    for mydir in [rawdir, processeddir]:
        os.makedirs(mydir, exist_ok=True)
    print(f"Processing {bookname}")
    rawjson = r.text.split(',"pageCount"')[0].split(',"pages":')[1]
    rawlist = json.loads(rawjson)
    for mypage in tqdm(rawlist[:]):
        targetpage = mypage['page'].zfill(3)
        targetfilename = rawdir + targetpage + ".jpg"
        targeturl = urlpre + urllib.parse.unquote(mypage['identifier']) + urlpost
        if not os.path.exists(targetfilename):
            r = requests.get(targeturl, headers=headers)
            if r.status_code == 200:
                with open(targetfilename, "wb") as outfile:
                    outfile.write(r.content)

100%|█████████████████████████████████████████████████████████████████████████████| 330/330 [00:00<00:00, 12228.94it/s]

Processing Livro de batismos de livres, 1774-1787


In [14]:
for bookdir in tqdm(list(glob("books/*"))):
    print(f"Processing {bookdir}")
    rawdir = bookdir + "/raw/"
    processeddir = bookdir + "/processed/"
    rawfiles = glob(rawdir + "*.*")
    for rawfile in tqdm(rawfiles):
        rawfile = rawfile.replace("\\", "/")
        targetfile = rawfile.replace("/raw/", "/processed/")
        if not os.path.exists(targetfile):
            im = Image.open(rawfile)
            if im.width > im.height:
                im = im.rotate(90)
            im = ImageOps.autocontrast(im, cutoff=autocontrastcutoff)
            enhancer = ImageEnhance.Sharpness(im)
            im = enhancer.enhance(imagesharpness)
            im.save(targetfile)


  0%|                                                                                          | 0/330 [00:00<?, ?it/s]

Processing books\Livro de batismos de livres, 1774-1787



 20%|████████████████▏                                                                | 66/330 [00:34<02:23,  1.85it/s]


 40%|████████████████████████████████                                                | 132/330 [01:06<01:35,  2.08it/s]


 60%|████████████████████████████████████████████████                                | 198/330 [01:39<01:10,  1.86it/s]


 80%|████████████████████████████████████████████████████████████████                | 264/330 [02:10<00:30,  2.14it/s]


100%|████████████████████████████████████████████████████████████████████████████████| 330/330 [02:40<00:00,  2.05it/s]


100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [02:40<00:00, 160.98s/it]


In [ ]:
rawfiles